<a href="https://colab.research.google.com/github/roni9826/Colab-Error-Example/blob/main/%5BMAKE_A_COPY%5D%5BV2_1%5D_Automated_Results_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###How to use?

The user must provide the following:
- Select your project ID you use to run querys within BQ. It is right below this section. If you see a 'Hidden cell', clicking on that would make the project id dropdown visible.
- Once that's done, click in "Runtime" and "Run all".
- Go to the **Deploy Application** section in the Table of contents and you'll find that the application will open up below the last code block.

**If you wish to analyze multiple tests, click Reset before selecting a new test. That will clear the preset conditions and data stored in the notebook.**

In [ ]:
PROJECT_ID = "logistics-customer-staging" # @param ["logistics-customer-staging", "dhh---analytics-eu","dhh---analytics-apac","dhub-yemek","peya-food-and-groceries","peya-delivery-and-support","peya-argentina"]
DATASET="curated_data_shared"

# Code

## Imports

In [ ]:
!pip install --upgrade plotly -q
!pip install pandas -q
!pip install polars -q
!pip install jupyter-dash -q
!pip install dash-bootstrap-components -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
import os
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Union, Callable, Optional, Tuple
import logging
import math

import numpy as np
import pandas as pd
import polars as pl
from dash import Dash, Input, Output, State, ctx, dash_table, dcc, html, no_update
from dash.exceptions import PreventUpdate
from google.cloud import bigquery
from jupyter_dash import JupyterDash
from dash_bootstrap_components import themes
import dash_bootstrap_components as dbc
from dash.dash_table.Format import Format, Symbol
from plotly import graph_objects as go

# power module
from statsmodels.stats.power import tt_ind_solve_power
from dataclasses import asdict, dataclass
from scipy.stats import norm

# GSheet handling
from google.auth import default
import gspread
from gspread_dataframe import set_with_dataframe

# Import packages for SBT module
import scipy.stats as stats
import statsmodels.formula.api as smf
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
import string

#Automated Results Tool
import time
import json
import concurrent.futures
from dash import exceptions as exc
from scipy.stats import gaussian_kde
from dash.dependencies import MATCH, ALL

## Common

### Constants

In [ ]:
BIGQUERY_CONFIG = {
    "project": PROJECT_ID,
    "dataset": DATASET,
}

### Context switcher

In [ ]:
class UnsupportedDeplomentModeException(BaseException):
    pass

class DeploymentModeNotImplemented(BaseException):
    pass


class DeploymentMode:
    LOCAL = "LOCAL"
    JUPYTER = "JUPYTER"
    COLAB = "COLAB"

    @classmethod
    def list_deployment_modes(cls):
        return [cls.LOCAL, cls.JUPYTER, cls.COLAB]


class BigQueryContextSwitcher(DeploymentMode):
    """
    This class initializes the BigQuery Client depending the environment the App is going to be deployed.
    This class is needed because the logic to authenticate in Local/Colab environment are different and we'd
    like to maintain support for both.
    """
    def __init__(self, env:str, project, *args, **kwargs) -> bigquery.Client:

        if not env in self.list_deployment_modes():
            raise UnsupportedDeplomentModeException("Unsupported deployment mode")

        self.env = env

        if self.env == self.LOCAL:
            # print(kwargs)
            # print("Local BigQueryClient .py deployment")
            self._get_bq_client_local(project, *args, **kwargs)
            self.file_output = ".xlsx"

        if self.env == self.COLAB:
            # print("BigQueryClient Google Colab deployment")
            # print(kwargs)
            self._get_bq_client_colab(project, *args, **kwargs)
            self._creds, _ = default()
            self._gc = gspread.authorize(self._creds)
            self.file_output = ""


        if self.env == self.JUPYTER:
            # print("Local BigQueryClient .py deployment")
            self._get_bq_client_local(project, *args, **kwargs)
            self.file_output = ".xlsx"


    def _get_bq_client_local(self, project:str, json_credentials:str) -> None:
        """Initialize a BigQuery Client using local credentials.
        This method requires a path to the json file that stores the account credentials

        Args:
            project (str): Billing Google project id to run queries
            json_credentials (str): filepath to account credentials

        Returns:
            bigquery.Client: _description_
        """
        self.json_credentials = json_credentials
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = self.json_credentials
        self.bq_client = bigquery.Client(project=project)

    def _get_bq_client_colab(self, project:str) -> None:
        """Initialize a BigQuery Client using Google Colab Auth service.
        Only the project id to run queries is required

        Args:
            project (str): _description_
        """
        from google.colab import auth, drive
        auth.authenticate_user()
        print('Authenticated')
        drive.mount('/content/gdrive')
        self.bq_client = bigquery.Client(project=project)
        # set the working directory to the user gdrive
        os.chdir("/content/gdrive/MyDrive")

    def export_single_df(self, df: pd.DataFrame, filename:str, sheet_name:str):
        """Write a DataFrame to either GSheets / Excel depending the run environment

        Args:
            df (pd.DataFrame): DataFrame to persist
            filename (str): filename to create
            sheet_name (str): filename's tab to persist dataframe
        """
        if self.env == self.LOCAL:
            print("Local Dash .py export")
            self._export_to_excel(df,  filename, sheet_name)

        if self.env == self.COLAB:
            print("Jupyter Dash Google Colab export")
            # print(kwargs)
            self._export_to_gsheets(df,  filename, sheet_name)

        if self.env == self.JUPYTER:
            print("Local Dash Jupyter export")
            self._export_to_excel(df, filename, sheet_name)

    def _export_to_excel(self, df: pd.DataFrame, filename:str, sheet_name:str):
        """Write a Dataframe to a Excel file.
        We first check if the file exists, if it does, then we need to append the dataframe
        to a new tab; otherwise, we create the file and write the DataFrame

        Args:
            df (pd.DataFrame): _description_
            filename (str): _description_
            sheet_name (str): _description_
        """
        if os.path.exists(filename) == False:
            df.to_excel(filename, sheet_name=sheet_name)
        else:
            with pd.ExcelWriter(filename, mode="a") as writer:
                df.to_excel(writer, sheet_name=sheet_name)
        print("Export Succesfull!")

    def _export_to_gsheets(self, df:pd.DataFrame, filename:str, sheet_name:str):
        """Export a DF to a GSheets
        We first try to open the file, if the file does not exists an error is returned
        which indicates that we must first create the file.
        After the file has been created/opened, the entire DF is pasted into a sheet
        Args:
            df (pd.DataFrame): _description_
            filename (str): _description_
            sheet_name (str): _description_
        """
        try:
            print(f"Opening {filename}...")
            file_obj = self._gc.open(filename)
        except:
            print(f"{filename} has not been created")
            print(f"Creating file {filename}")
            self._gc.create(filename)
            file_obj = self._gc.open(filename)
        sheet_obj = file_obj.add_worksheet(sheet_name, 2, 2)
        set_with_dataframe(sheet_obj, df)


class DashContextSwitcher(DeploymentMode):
    """
    This class initializes the Dash App depending the environment the App is going to be deployed.
    This class is needed because the logic to authenticate in Local/Colab environment are different and we'd
    like to maintain support for both.
    """
    def __init__(self, env:str, name:str, *args, **kwargs):
        if not env in self.list_deployment_modes():
            raise UnsupportedDeplomentModeException("Unsupported deployment mode")

        if env == self.LOCAL:
            # print("Local Dash .py deployment")
            self._get_dash_local(name, *args, **kwargs)

        if env == self.COLAB:
            # print("Jupyter Dash Google Colab deployment")
            # print(kwargs)
            self._get_dash_jupyter(name, *args, **kwargs)

        if env == self.JUPYTER:
            # print("Local Dash Jupyter mode deployment")
            self._get_dash_jupyter(name, *args, **kwargs)

    def _get_dash_local(self, name:str, *args, **kwargs):
        """Initialize a Dash App

        Args:
            name (str): _description_
        """
        self.dash_app = Dash(name, *args, **kwargs)

    def _get_dash_jupyter(self, name:str, *args, **kwargs):
        """Initialize a JupyterDash App

        Args:
            name (str): _description_
        """
        self.dash_app = Dash(name, *args, **kwargs)


### BigQuery

In [ ]:



class BaseBigQueryConnector(BigQueryContextSwitcher):
    """
    Class purpose is to handle connection to BQ and I/O operations
    """
    def __init__(self, env:str, project:str, dataset:str, *args, **kwargs) -> None:
        super().__init__(env, project, *args, **kwargs)
        self.dataset = dataset

    def run_query(self, query: str) -> bigquery.job.QueryJob:
        """Run a query in BigQuery a wait for its result

        Args:
            query (str): query to run

        Returns:
            bigquery.job.QueryJob:
        """
        job = self.bq_client.query(query)
        job.result()
        # self.logger.info("Query ran successfully")
        return job

    def get_df_from_query(self, query: str, type:str="pandas") -> Union[pd.DataFrame, pl.DataFrame]:
        """Generate a dataframe from the results of a query

        Args:
            query (str): _description_

        Returns:
            pd.DataFrame: _description_
        """
        job = self.run_query(query)

        if type == "pandas":
            return job.to_dataframe(progress_bar_type="tqdm")

        if type == "polars":
            return pl.from_arrow(job.to_arrow(progress_bar_type="tqdm"))

        raise Exception("Dataframe type not supported")




### Tab

In [ ]:
class BaseAdapter:
    pass


class TabModule:
    def __init__(self, module_name:str
                 , env:str
                 , dash: Dash
                 , layout: callable
                 , callbacks: callable
                 , adapter: BaseAdapter
        ):
        self.module_name = module_name
        self.env = env
        self.dash = dash
        self.callbacks = callbacks
        self.layout = layout
        self.adapter = adapter


    def get_tab(self):
        self.callbacks(self.dash, self.adapter)
        return dbc.Tab(label=self.module_name, children=[self.layout(self.adapter)], id=f"{self.module_name}-tab")

### Components

In [ ]:
def get_submit_feedback_button(prefix_id:str) -> dbc.Button:
    return (
        dbc.Button(
        "Give us feedback!"
        , id=f"{prefix_id}-summary-feedback-button"
        , color="link"
        , href="https://docs.google.com/forms/d/e/1FAIpQLSc4d8YjJe44HzG2Qr4NJrvwdIxiIJnfMM_cC2gCuAMwBGC3oA/viewform"
        , external_link=True
        )
    )


def get_date_picker_component(id:str) -> html.Div:
    """Generate a generic single date picker
    with a default date

    Args:
        id (str): _description_

    Returns:
        html.Div: _description_
    """
    return (
        html.Div(
            children=[
                html.Label("Date")
                , dcc.DatePickerSingle(
                    id=id
                    , date = (datetime.now() - timedelta(days=3)).date()
                )
            ]
        )
    )

### Util

In [ ]:
class Logger:
    @staticmethod
    def log_gen(logger_name:str) -> logging.Logger:
        # set logger
        logger = logging.getLogger(logger_name)
        logger.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')


        # # set file handler
        # handler = logging.FileHandler(log_file_path)
        # handler.setLevel(logging.INFO)
        # handler.setFormatter(formatter)


        #stream handler
        stream_handler = logging.StreamHandler()
        stream_handler.setLevel(logging.INFO)
        stream_handler.setFormatter(formatter)

        # add handlers to logger
        logger.addHandler(stream_handler)
        return logger

    @staticmethod
    def reset_logger(logger_name:str):
        logger = logging.getLogger(logger_name)

        # Remove all handlers from the logger
        for handler in logger.handlers:
            logger.removeHandler(handler)

        # Remove all filters from the logger
        for filter_ in logger.filters:
            logger.removeFilter(filter_)


#################################### UTILITIES #########################################

def get_unique_list_from_polars_df(df:pl.DataFrame, column:str):
    """Returns a list with the unique elements of a polars dataframe.
    We need first need to convert the polars column to numpy. It returns
    a 2D array, so we need to flatten it to get a 1D array using ravel().

    Args:
        df (pl.DataFrame): Polars dataframe
        column (str): column to get unique values from
    """
    return (
        df
        .select(column)
        .unique()
        .to_numpy()
        .ravel()
        .tolist()
    )

def cast_to_list_from_polars_column(df:pl.DataFrame):
    """Cast a polars dataframe to a list. This is useful when we want to
    return a list of values from a single column in a polars dataframe.
    We need first to convert the polars dataframe to a series and then use
    the to_list() method.

    Args:
        df (pl.DataFrame): Polars dataframe
        column (str): column to get unique values from
    """
    return (
        df
        .to_series()
        .unique()
        .to_list()
    )


def cast_input_to_list(input:any, clean_element_func: Optional[callable] = None) -> list:
    """
    Transforms the given input into a list.

    If the input is already a list and a cleaning function is provided,
    apply it to each element of the list. If no cleaning function is provided, the
    list is returned as is. If the input is not a list, it is encapsulated into a list. If the input is None,
    an empty list is returned.
    """

    if isinstance(input, list):
        if clean_element_func is not None:
            return [clean_element_func(item) for item in input]
        return input
    elif input is None:
        return []
    else:
        return [input]

## Init APP

In [ ]:
#################################### INIT APP
app_logger = Logger.log_gen("APP")

app_logger.info("Initializing App.....")
ENV = "COLAB"
app = DashContextSwitcher(ENV, __name__,  external_stylesheets=[themes.BOOTSTRAP
                                                                , dbc.icons.BOOTSTRAP
                                                                , 'https://fonts.googleapis.com/css2?family=Outfit:wght@100..900&display=swap'
                                                                ]
                          , suppress_callback_exceptions=True).dash_app
app_logger.info(f"ENV is {ENV}.....")

2025-01-22 15:38:45,609 - INFO - Initializing App.....
INFO:APP:Initializing App.....
2025-01-22 15:38:45,662 - INFO - ENV is COLAB.....
INFO:APP:ENV is COLAB.....


## Report Tool

### config.py

In [ ]:

#####QUERIES
EXP_ENTITIES_QUERY = """
SELECT
entity_id,
test_id,
test_name,
hypothesis,
zone_ids,
test_start_date,
test_end_date,
is_already_executed,
misconfigured,
test_vertical_parents,
is_active,
variation_group,
variation_share,
customer_condition,
customer_areas,
vendor_filters
FROM `fulfillment-dwh-production.{dataset}.dps_experiment_setups`
WHERE DATE(test_start_date) > DATE('2023-06-01')
AND misconfigured is false
AND is_already_executed is true
ORDER BY test_start_date desc
"""

AVAILABLE_GROUPINGS_QUERY = """
DECLARE test STRING DEFAULT '{test_name_filter}';
DECLARE entity STRING DEFAULT '{entity_id_filter}';
DECLARE country STRING DEFAULT LOWER((SPLIT(entity,'_'))[OFFSET(1)]);

WITH distinct_zones AS (
  Select distinct zone_id
  FROM `fulfillment-dwh-production.{dataset}.dps_experiment_setups`, UNNEST(zone_ids) AS zone_id
  WHERE test_name = test
  AND entity_id = entity
),
 zone_names AS (
  SELECT DISTINCT
  test_name,
   zone_name
  FROM `fulfillment-dwh-production.{dataset}.dps_test_orders`
  WHERE test_name = test
  AND entity_id = entity
  AND zone_id IN (SELECT * FROM distinct_zones)
),
customer_loc_id AS (
  SELECT
  DISTINCT
  test_name,
  cust_area.id
  FROM `fulfillment-dwh-production.{dataset}.dps_experiment_setups`, UNNEST(customer_areas) AS cust_area
  WHERE test_name = test
  AND entity_id = entity
),
customer_locations AS (
  SELECT
  area_id,
  cah.area_name
  FROM `fulfillment-dwh-production.curated_data_shared.pricing_customer_area_versions`, UNNEST(customer_area_history) AS cah
  WHERE area_id IN (SELECT id from customer_loc_id)
  AND country_code = country
),

city_names AS (
  SELECT
  DISTINCT
  test_name,
  city_name
  FROM `fulfillment-dwh-production.{dataset}.dps_test_orders`
  WHERE test_name = test
  AND entity_id = entity
),

time_of_day AS (
  SELECT
  DISTINCT
  CONCAT(CAST(TIME(schedule.start_at) AS STRING), ' to ', CAST(TIME(schedule.end_at) AS STRING)) as schedule_time
  FROM `fulfillment-dwh-production.{dataset}.dps_experiment_setups`
  WHERE test_name =test
  AND schedule.id is not null
  ORDER BY schedule_time
),

target_groups AS (
  SELECT
  DISTINCT target_group
  FROM `fulfillment-dwh-production.{dataset}.dps_test_orders`
  WHERE test_name = test
  AND entity_id = entity
  ORDER BY target_group
)

SELECT
  'Target Groups' AS category,
  STRING_AGG(DISTINCT target_group, ', ') AS list_of_elements
FROM target_groups
UNION ALL
SELECT
  'Time of Day' AS category,
  STRING_AGG(DISTINCT schedule_time, ', ') AS list_of_elements
FROM time_of_day
UNION ALL
SELECT
  'City' AS category,
  STRING_AGG(DISTINCT city_name, ', ') AS list_of_elements
FROM city_names
UNION ALL
SELECT
  'Customer Location' AS category,
  STRING_AGG(DISTINCT area_name, ', ') AS list_of_elements
FROM customer_locations
UNION ALL
SELECT
  'Zone' AS category,
  STRING_AGG(DISTINCT zone_name, ', ') AS list_of_elements
FROM zone_names
"""

TEST_DATA_QUERY = """
DECLARE test STRING DEFAULT '{test_name_filter}';
DECLARE entity STRING DEFAULT '{entity_id_filter}';
DECLARE country STRING DEFAULT LOWER((SPLIT(entity,'_'))[OFFSET(1)]);

WITH date_filter AS (
  SELECT DISTINCT
  DATE_SUB(DATE(test_start_date), INTERVAL 1 DAY) AS test_start,
  IFNULL(DATE(test_end_date), CURRENT_DATE())  AS test_end
  FROM `fulfillment-dwh-production.{dataset}.dps_experiment_setups`
  WHERE test_name= test
  AND entity_id = entity
)

, load_test_order AS (

  SELECT
  -- order info
  platform_order_code,
  entity_id,
  --perseus_client_id,
  test_variant,
  -- granularity info
  target_group,
  zone_name,
  city_name,
  order_day,
  order_hour,
  -- dps travel + fees info
  dps_travel_time_fee_local,
  dps_delivery_fee_local,
  -- cpo/gfv/gmv/profit/rev info
  gmv_local,
  profit_local,
  revenue_local,
  gmv_eur,
  profit_eur,
  revenue_eur,
  has_subscription,
  has_new_customer_condition
  FROM `fulfillment-dwh-production.{dataset}.dps_test_orders`
  WHERE created_date  >= (SELECT test_start FROM date_filter)
  AND entity_id = entity
  AND test_name = test
  AND is_sent is True
)

, load_dps_order As (
  SELECT
  platform_order_code,
  entity_id,
  order_placed_at_local,
  travel_time_distance_km as tt_dist_km,
  is_in_treatment,
  customer_location,
  conditions.customer_area_id as customer_area_id,
  TRUNC(dps_travel_time) + (dps_travel_time - TRUNC(dps_travel_time))*60/100 as dps_travel_time_min,
  FROM `fulfillment-dwh-production.{dataset}.dps_sessions_mapped_to_orders`
  WHERE created_date >= (SELECT test_start FROM date_filter)
  AND created_date <= (SELECT test_end FROM date_filter)
  AND entity_id = entity

)

, customer_area_names AS (
  SELECT
  area_id,
  cah.area_name
  FROM `fulfillment-dwh-production.{dataset}.pricing_customer_area_versions` , UNNEST (customer_area_history) AS cah
  WHERE country_code = country
  AND cah.active_to IS NULL
)

SELECT t.*
, o.* EXCEPT(platform_order_code, entity_id), cah.area_name
FROM load_test_order t
LEFT JOIN load_dps_order o
ON t.entity_id = o.entity_id
AND t.platform_order_code = o.platform_order_code
LEFT JOIN customer_area_names cah
ON o.customer_area_id = cah.area_id
"""

LOCAL_CURRENCY_QUERY = """
DECLARE test STRING DEFAULT '{test_name_filter}';
DECLARE entity STRING DEFAULT '{entity_id_filter}';
DECLARE country STRING DEFAULT LOWER((SPLIT(entity,'_'))[OFFSET(1)]);
SELECT
  currency_code
  FROM `fulfillment-dwh-production.{dataset}.countries`
  WHERE country_code = country
"""

EXP_SETUP_QUERY ="""
DECLARE test STRING DEFAULT '{test_name_filter}';
DECLARE entity STRING DEFAULT '{entity_id_filter}';

SELECT
DISTINCT
test_name,
objective,
hypothesis,
DATE(test_start_date) as start_date,
DATE(test_end_date) as end_date,
variation_group,
variation_share,
DATE(updated_at) as updated_date
 FROM `fulfillment-dwh-production.{dataset}.dps_experiment_setups`
 WHERE entity_id = entity
 AND test_name = test
 ORDER BY updated_date DESC
 """

SIGNIFICANCE_QUERY = """
DECLARE test STRING DEFAULT '{test_name_filter}';

SELECT
test_name,
kpi_label,
group_a as variant_a,
group_b as variant_b,
CASE
  WHEN statistical_method = 'cuped_t_test' THEN cuped_mean_a
  WHEN statistical_method = 'delta' THEN mean_a
END AS mean_a,
CASE
  WHEN statistical_method = 'cuped_t_test' THEN cuped_mean_b
  WHEN statistical_method = 'delta' THEN mean_b
END AS mean_b,
CASE
  WHEN statistical_method = 'cuped_t_test' THEN corrected_cuped_p_value
  WHEN statistical_method = 'delta' THEN corrected_p_value
END AS p_value,
values_count_a,
values_count_b,
label as treatment,
CASE
  WHEN statistical_method = 'cuped_t_test' THEN 'CUPED'
  WHEN statistical_method = 'delta' THEN 'Delta'
END AS method
FROM `logistics-data-storage-staging.shared_pricing.dps_ab_test_significance_dataset`
WHERE test_name = test
AND group_a <> group_b
"""

TARGET_PPO_QUERY = """
DECLARE entity STRING DEFAULT '{entity_id_filter}';

SELECT
AVG(profit_local) AS target_ppo
FROM `fulfillment-dwh-production.{dataset}.dps_sessions_mapped_to_orders`
WHERE DATE(order_placed_at_local) > DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH)
AND entity_id = entity
AND vertical_type IN ('restaurants')
"""

####### GRANULARITY COLUMN NAMES
GRANULARITY_COL_NAMES= {
    "Target Groups": "target_group",
    "Zone": "zone_name",
    "City": "city_name",
    "Customer Location": "area_name",
    "Time of Day": "order_hour"
}

####### TOTAL KPIs
TOTAL_KPIS = [
    "Orders",
    "Revenue",
    "Profit",
    "GMV"
]

#COLORS
COLORS = [
    '#D61F26'
    ,'#4629FF'
    ,'#00D61D'
    ,'#C7C7C7'
    ,'#8638E0'
]

#SIGNIFICANT KPI ORDER
SIG_KPIS = {
    #P&L
    "profit_local":"Profit per Order",
    "profit_local_per_user":"Profit per User",
    "orders_per_user":"Orders per User",
    "revenue_local_per_user":"Revenue per User",
    "gmv_local_per_user":"GMV per User",
    #CVR
    "cvr": "Conversion Rate",
    "cvr3": "Conversion Rate 3",
    "mcvr2": "mCVR 2",
    "mcvr3": "mCVR 3",
    "mcvr4": "mCVR 4",
    #FEES
    "commission_local_per_user":"Commission per User",
    "dps_delivery_fee_local_per_user":"DPS Delivery Fee per User",
    "service_fee_local_per_user":"Service Fee per User",
    "mov_customer_fee_local_per_user":"MOV Customer Fee per User",
    #LOGISTICS
    "travel_time_per_user":"Travel Time per User",
    "delivery_distance_per_user":"Delivery Distance per User",
    "delivery_costs_local_per_user":"Delivery Costs Local per User"
}

EXCLUDE = "exc"
INCLUDE = "inc"
EUR= "EUR"


############PRICING ACTION STANDARDS

# 1. CHECKS
PAC_CHECKS_INCREASE= [
    "1. Change needs to have a positive return on investment",
    "2. Understand likelihood of a negative impact",
    "3. Limit lost orders and recuperate if need be",
    "4. Ensure that the long term benefit is also worth it"
]

PAC_CHECKS_DECREASE = [
    "1. Change needs to have a positive return on investment",
    "2. Understand likelihood of a negative impact",
    "3. Compare with other marketing metrics",
    "4. Ensure that the long term benefit is also worth it"
]


# 2. THEORY
PAC_THEORY_INCREASE = [
    "ROI > 1",
    "At what order change is ROI <1",
    "Limit investment needed",
    "Long term profit targets"
]

PAC_THEORY_DECREASE = [
    "ROI > 1",
    "At what order change is ROI < 1",
    "Make the best choice",
    "Long term profit targets"
]

# 3. COMPARISONS
PAC_COMPARE_INCREASE = [
    " PPOL > Control Profit per order",
    "% Delta orders < AOD",
    "Recoup ROI < Accepted ROI",
    "PPOL > Target Profit per order"
]

PAC_COMPARE_DECREASE = [
    "CPIO < Control Profit per order",
    "% Delta orders > AOD",
    "Pricing vs Marketing ROI",
    "CPIO < Target Profit per order"
]

######## Query Input Models
@dataclass
class TestQueryInputs:

    entity_id_filter: str
    test_name_filter: str

    def test_query_dict(self):
        return asdict(self)

@dataclass
class GroupParameters:
    group_name: str
    inc_criteria: str
    groupings: list[str]

    def group_parameter_dict(self):
        return asdict(self)

@dataclass
class ReportParameters:
    entity_id: str
    test_name: str
    treatment_scope: str
    selected_granularity: str
    granularity_groups: list[GroupParameters]

    def report_parameter_dict(self):
        return asdict(self)



### backend.py

In [ ]:
########## STORE ##########
class Store:
    """
    Database once the app is loaded. Local instance of BigQuery data.
    How does it interact with other classes/functions?
    """

    """
    CLASSES
    """
    test_query_inputs: TestQueryInputs = None       # Class containing test name and entity id
    report_parameters_dict: ReportParameters = None      # Class containing Report Parameters (Granularity)

    """
    DATAFRAMES
    """
    entities_data: pd.DataFrame = None              # Dataframe containing the entities and tests that the user has access to
    df_groupings_from_bq: pd.DataFrame = None       # Dataframe containing the potential groupings
    df_test_data: pd.DataFrame = None               # Dataframe containing all the test data
    df_total_kpis: pd.DataFrame = None              # Dataframe containing the total KPI info
    df_significance_data: pd.DataFrame = None       # Dataframe containing signficance table data
    df_data_with_groupings: pd.DataFrame = None
    df_cpio_ppol: pd.DataFrame = None               # Dataframe containing cpio/ppol information
    df_pac: pd.DataFrame = None                     # Dataframe containing pac information. Gets dynamically changed when variation is changed in Pricing Action Standards. Should only have one row of data.

    """
    DICTIONARIES
    """
    experiment_overview: dict = None                # Dataframe containing all the experiment setup

    """
    LISTS
    """
    available_granularities: list[str] = None       # List of Strings for available grouping dropdown
    available_data_groupings: list[int] = None      # List of Strings for available data group numbers
    curr_cd_list:list[str] = None                        # Currency Code

    """
    CONSTANTS
    """
    no_granularity = "Analyze test without granularity"
    chosen_granularity: str = "Analyze test without granularity"
    selected_group_count: int = None
    treatment_scope: str = "All"                    # Treatment Scope
    subscribers:str = INCLUDE                       # Include/ Exclude Subscription orders
    fdnc:str = INCLUDE                              # Include/ Exclude FDNC
    curr_cd: str                                    # Selected Currency Code
    test_objective:str = ""                         # Test Objective
    ppo_ctrl:float = None                           # Profit per Order of Control
    pas_flow:str = ""                               # Pricing Action Standards flow. Possible values: increase_fees, decrease_fees
    target_ppo:float = None                         # Target PPO Value
    curr_conv_ratio: float = None                   # Currency Converter Ratio


    #Gets all the entities that the user has access to
    #This should be called when the app is being loaded
    def fetch_entities(self) -> list[str]:
        return self.entities_data["entity_id"].sort_values().unique().tolist()

    def fetch_tests(self,entity_id) -> list[str]:

        return (
          self.entities_data[self.entities_data["entity_id"] == entity_id]["test_name"]
          .sort_values(ascending=False)
          .unique()
          .tolist()
        )

class ResultsToolConnector(BaseBigQueryConnector):
    """
    What is the purpose of this class?
    How does it interact with other classes/functions?
    """

    #Constructor
    def __init__(self, env:str, project:str, dataset:str, *args, **kwargs) -> None:
        super().__init__(env, project, dataset, *args, **kwargs)


    def fetch_entities_data(self) -> pd.DataFrame:
        """Load the entities available in the dataset

        Returns:
            pd.DataFrame: _description_
        """
        query = EXP_ENTITIES_QUERY.format(dataset=self.dataset)
        return self.get_df_from_query(query)


class ResultsToolAdapter(BaseAdapter):
    """
    What is the purpose of this class?
    How does it interact with other classes/functions?
    """

    #Constructor
    def __init__(self, connector: ResultsToolConnector) -> None:
        """
        Constructor that is called when the tool is initialized
        """
        self.connector = connector
        self.store = Store()
        self.logger = Logger.log_gen("AutomatedExperimentResultsAdapter")
        self.logger.info("AER Adapter initialized")

        self.load_entities()

    def load_entities(self) -> list[str]:
        """
        Load the entities available to the user.
        """
        self.store.entities_data = self.connector.fetch_entities_data()
        self.logger.info("AER Entities loaded")

    #GRANULARITY AND GROUPING
    def fetch_grouping_options(self,test_query_input:TestQueryInputs) -> pd.DataFrame:
        """
        Invoked by:

        Returns:
            A dataframe with the following columns:
                Category : Available groupings category for this particular test. egs. Zone, Customer Location etc.
                List of Elements : Comma separated string of all the unique values for the particular category
        """
        query_inputs = test_query_input.test_query_dict() | {"dataset": self.connector.dataset}
        return self.connector.get_df_from_query(AVAILABLE_GROUPINGS_QUERY.format(**query_inputs))

    def get_granularities_with_multiple_elements(self, df_granularities) -> list[str]:
        """
        Invoked by:
            load_test_groupings_into_store
        Returns:
            A list of available granularities for the chosen graularity.

        """
        df_granularities['num_elements']= df_granularities['list_of_elements'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)
        available_granularities = df_granularities[df_granularities['num_elements']>1]['category'].tolist()
        available_granularities.append(self.store.no_granularity)
        return available_granularities

    def load_test_groupings_into_store(self, test_query_inputs:TestQueryInputs):
        """
        Invoked by:

        Function:
        """
        self.logger.info("Function: load_test_grouping_into_store")
        self.store.df_groupings_from_bq = self.fetch_grouping_options(test_query_inputs)
        self.store.test_query_inputs = test_query_inputs
        self.store.available_granularities = self.get_granularities_with_multiple_elements(self.store.df_groupings_from_bq)

    def populate_data_group_count_options(self, max_count: int) -> list[str]:
        """
        Invoked by:

        Function:
        """
        data_group_opts =  list(range(2,max_count+1))
        return data_group_opts

    def generate_data_group_count_options(self,chosen_granularity:str):
        """
        Invoked by:

        Function:
        """
        df_groupings = self.store.df_groupings_from_bq
        max_count = df_groupings.loc[df_groupings['category']==chosen_granularity, 'num_elements'].values[0]
        self.store.available_data_groupings = self.populate_data_group_count_options(max_count)


    #Load Order Data
    def fetch_test_order_data(self, test_query_input:TestQueryInputs) -> pd.DataFrame:
        """
        Invoked by load_test_data_into_store and uses results_tool_connector to connect to BQ and load test orders
        Invoked by:

        Function:
        """
        query_inputs = test_query_input.test_query_dict() | {"dataset": self.connector.dataset}
        return self.connector.get_df_from_query(TEST_DATA_QUERY.format(**query_inputs))


    def load_test_data_into_store(self, test_query_inputs:TestQueryInputs):
        """
        Invoked from the front end and calls the fetch_test_order_data function
        Invoked by:

        Function:
        """
        self.logger.info("Function: load_test_data_into_store")
        self.store.df_test_data= self.fetch_test_order_data(test_query_inputs)

    def fetch_currency_cd(self, test_query_input:TestQueryInputs):
        """
        Invoked by load_currency_code_into_store and connects to DB with the query LOCAL_CURRENCY_QUERY
        Invoked by:

        Function:
        """
        query_inputs = test_query_input.test_query_dict() | {"dataset": self.connector.dataset}
        df_curr =  self.connector.get_df_from_query(LOCAL_CURRENCY_QUERY.format(**query_inputs))
        self.logger.info(df_curr)
        return df_curr['currency_code'].iloc[0]

    def load_currency_code_into_store(self,test_query_inputs:TestQueryInputs):
        """
        Invoked from the front end when load test is clicked.
        Invoked by:

        Function:
        """
        self.logger.info("Function: load_test_currency_into_store")
        local_curr_cd = self.fetch_currency_cd(test_query_inputs)
        list_of_available_currency = [local_curr_cd, EUR]
        self.store.curr_cd_list = list_of_available_currency
        self.store.curr_cd = EUR

    def update_selected_currency(self, selected_currecy):
        self.store.curr_cd = selected_currecy

    def load_significance_data_into_store(self, test_query_input:TestQueryInputs):
        """
        Invoked by:

        Function:
        """
        self.logger.info("Function: load_significance_data_into_store")
        self.store.df_significance_data = self.fetch_significance_data(test_query_input)


    def fetch_significance_data(self, test_query_input:TestQueryInputs) ->pd.DataFrame:
        """
        Invoked by:

        Function:
        """
        query_inputs = test_query_input.test_query_dict() | {"dataset":self.connector.dataset}
        df_significance = self.connector.get_df_from_query(SIGNIFICANCE_QUERY.format(**query_inputs))

        #GET PROFIT PER ORDER CONTROL AND STORE TO USE LATER FOR PRICING ACTION STANDARDS
        filtered_ppo_ctrl = df_significance.loc[
            (df_significance['test_name'] == test_query_input.test_name_filter) &
            (df_significance['variant_a'] == 'Control') &
            (df_significance['kpi_label'] == 'profit_local') &
            (df_significance['treatment'] == 'All')
        ]
        mean_a_value = filtered_ppo_ctrl['mean_a'].values[0] if not filtered_ppo_ctrl.empty else None
        self.logger.info("PPO Ctrl: "+ str(mean_a_value))
        self.store.ppo_ctrl = mean_a_value
        return df_significance

    def load_target_ppo(self, test_query_input:TestQueryInputs):
        self.logger.info("Function: load_target_ppo")
        target_ppo = self.fetch_target_ppo(test_query_input)
        self.store.target_ppo = target_ppo

    def fetch_target_ppo(self, test_query_input:TestQueryInputs) -> float :
        """
        Invoked by:

        Function:
        """

        query_inputs = test_query_input.test_query_dict() | {"dataset":self.connector.dataset}
        df_target_ppo = self.connector.get_df_from_query(TARGET_PPO_QUERY.format(**query_inputs))
        target_ppo = float(df_target_ppo.iloc[0,0])
        self.logger.info("Target PPO from fetch: "+ str(target_ppo))
        return target_ppo

    def populate_granularity_options_into_input_dropdowns(self, chosen_granularity) -> list[str]:
        """
        Invoked by:

        Function:
        """
        df_groupings = self.store.df_groupings_from_bq
        granularity_opts = df_groupings.loc[df_groupings['category']==chosen_granularity, 'list_of_elements'].values[0]
        options_list = granularity_opts.split(',')
        return options_list

    def update_orders_with_exclusions(self,subscribers, fdnc):
        if subscribers:
            self.store.subscribers = EXCLUDE
        else:
            self.store.subscribers = INCLUDE

        if fdnc:
            self.store.fdnc = EXCLUDE
        else:
            self.store.fdnc = INCLUDE

        self.logger.info("Subscriber status: " + self.store.subscribers)
        self.logger.info("FDNC Status: "+ self.store.fdnc)

    def create_report_parameter(self,group_parameter_list=None) -> ReportParameters: # type: ignore
        """
        Invoked when the granularity groups have been created.
        Invoked by:

        Function:
        """

        if group_parameter_list is None:
            group_parameter_list = []

        report_parameters = ReportParameters(
            entity_id=self.store.test_query_inputs.entity_id_filter,
            test_name=self.store.test_query_inputs.test_name_filter,
            treatment_scope=self.store.treatment_scope,
            selected_granularity=self.store.chosen_granularity,
            granularity_groups=group_parameter_list
        )

        self.store.report_parameters_dict = report_parameters

        #self.store.report_parameters = report_parameters
        self.logger.info(str(self.store.report_parameters_dict))
        #self.logger.info(self.store.curr_cd)

    def fetch_test_overview_df(self,test_query_input: TestQueryInputs) -> pd.DataFrame:
        """
        Invoked by:

        Function:
        """
        query_inputs = test_query_input.test_query_dict() | {"dataset": self.connector.dataset}
        df = self.connector.get_df_from_query(EXP_SETUP_QUERY.format(**query_inputs))
        return df

    def get_test_overview_card_data(self):
        """
        Invoked by:

        Function:
        """
        df_experiment_setup = self.fetch_test_overview_df(self.store.test_query_inputs)

        objective = df_experiment_setup['objective']
        test_obj = objective.iloc[0]

        hypothesis = df_experiment_setup['hypothesis']
        test_hypo = hypothesis.iloc[0]

        test_start = df_experiment_setup['start_date']
        test_st_date = test_start.iloc[0]

        test_end = df_experiment_setup['end_date']
        test_end_date = test_end.iloc[0]

        df_variation = df_experiment_setup[['variation_group','variation_share']]
        df_unique_variation = df_variation.drop_duplicates(subset=['variation_group', 'variation_share'])
        variations_list = df_unique_variation['variation_group'].to_list()
        variation_share_list = df_unique_variation['variation_share'].to_list()
        self.store.test_objective = test_obj

        return test_obj, test_hypo, test_st_date, test_end_date, variations_list, variation_share_list


    def get_roi_of_each_variant(self) ->  str :
        """
        Invoked by:

        Function:
        """
        treatment_scope = self.store.report_parameters_dict.treatment_scope
        self.logger.info(treatment_scope)
        if treatment_scope == "True":
            df = self.store.df_test_data[self.store.df_test_data['is_in_treatment']==True]
        else:
            df = self.store.df_test_data

        if self.store.curr_cd == EUR:
            profit_column = 'profit_eur'
        else:
            profit_column = 'profit_local'


        df_grouped = df.groupby('test_variant').agg({profit_column:'sum'
                                                     , 'platform_order_code':'count'
                                                     })

        df_grouped['profit_per_order'] = df_grouped[profit_column]/df_grouped['platform_order_code']

        control_profit_per_order = df_grouped.loc['Control', 'profit_per_order']
        control_orders = df_grouped.loc['Control', 'platform_order_code']

        df_grouped['cpio/ppol'] = df_grouped.apply(lambda row: abs(row['profit_per_order'] - control_profit_per_order) * row['platform_order_code']
                                              / abs(row['platform_order_code'] - control_orders) if row.name != 'Control' else None, axis=1)

        df_grouped['cpio/ppol'] = df_grouped['cpio/ppol'].round(2)
        self.store.df_cpio_ppol = df_grouped
        result = df_grouped['cpio/ppol'].dropna().to_dict()
        result_json = json.dumps(result)
        return result_json

    def get_cpio_ppol_header(self):
        if self.store.test_objective == 'Profit':
            return 'PPoL'
        elif self.store.test_objective == 'Orders':
            return 'CPiO'
        else:
            return 'CPiO/PPoL'


    def filter_exclusions(self):
        df = self.store.df_test_data
        if self.store.subscribers == EXCLUDE:
            df =df[df['has_subscription'] == False]

        if self.store.fdnc == EXCLUDE:
            df = df[(df['has_new_customer_condition'] == False) & (df['dps_delivery_fee_local'] != 0)]

        self.store.df_test_data = df
        self.logger.info(len(df))

    def get_totals_table(self) -> pd.DataFrame:
        """
        Invoked by:

        Function:
        """
        treatment_scope = self.store.report_parameters_dict.treatment_scope

        if treatment_scope == "True":
            df = self.store.df_test_data[self.store.df_test_data['is_in_treatment']==True]
        else:
            df = self.store.df_test_data

        # Calculate metrics for each test_variant
        if self.store.curr_cd == EUR:
            df_grouped = df.groupby('test_variant').agg({
                'platform_order_code': 'count',
                'revenue_eur': 'sum',
                'profit_eur': 'sum',
                'gmv_eur': 'sum'
            }).reset_index()

            df_orders = df_grouped[['test_variant', 'platform_order_code']].rename(columns={'platform_order_code': 'Orders'}).set_index('test_variant').T
            df_revenue = df_grouped[['test_variant', 'revenue_eur']].rename(columns={'revenue_eur': 'Revenue'}).set_index('test_variant').T
            df_profit = df_grouped[['test_variant', 'profit_eur']].rename(columns={'profit_eur': 'Profit'}).set_index('test_variant').T
            df_gmv = df_grouped[['test_variant', 'gmv_eur']].rename(columns={'gmv_eur':'GMV'}).set_index('test_variant').T
        else:
            df_grouped = df.groupby('test_variant').agg({
                'platform_order_code': 'count',
                'revenue_local': 'sum',
                'profit_local': 'sum',
                'gmv_local': 'sum'
            }).reset_index()

            # Reshape the data
            df_orders = df_grouped[['test_variant', 'platform_order_code']].rename(columns={'platform_order_code': 'Orders'}).set_index('test_variant').T
            df_revenue = df_grouped[['test_variant', 'revenue_local']].rename(columns={'revenue_local': 'Revenue'}).set_index('test_variant').T
            df_profit = df_grouped[['test_variant', 'profit_local']].rename(columns={'profit_local': 'Profit'}).set_index('test_variant').T
            df_gmv = df_grouped[['test_variant', 'gmv_local']].rename(columns={'gmv_local':'GMV'}).set_index('test_variant').T

        # Concatenate all dataframes
        df_total = pd.concat([df_orders, df_revenue, df_profit, df_gmv])
        df_total.reset_index(inplace=True)
        df_total.rename(columns={'index': 'KPI'}, inplace=True)
        df_total = df_total.round(0)
        # Print the new DataFrame

        for col in df_total.columns[1:]:
            df_total[col] = df_total[col].astype(int)

        for col in df_total.columns[1:]:
            if col != 'Control':
                df_total['Delta ' + col] = df_total[col] - df_total['Control']
                df_total['% Delta '+ col] = 100*df_total['Delta '+col]/df_total['Control'].abs()


        for col in df_total.columns:
            if '% Delta' in col:

                df_total[col] = df_total[col].round(2).astype(str) + '%'


            elif 'KPI' not in col and '%Delta' not in col:
                df_total[col] = df_total[col].astype(int).apply(lambda x: '{:,}'.format(x))

        test_obj = self.store.test_objective
        roi_row = pd.DataFrame({'KPI':['ROI']})
        df_cpio_ppol = self.store.df_cpio_ppol
        df_cpio_ppol = df_cpio_ppol.reset_index()
        ppo_ctrl = self.store.ppo_ctrl

        if self.store.curr_cd == 'EUR':
            profit_local_sum = df['profit_local'].sum()
            profit_eur_sum = df['profit_eur'].sum()

            conv_ratio = profit_eur_sum/profit_local_sum
            self.store.curr_conv_ratio = conv_ratio
            ppo_ctrl = ppo_ctrl*conv_ratio

        if test_obj == 'Profit':
            df_total = pd.concat([df_total, roi_row], ignore_index=True)
            for index, row in df_cpio_ppol.iterrows():
                test_variant = row['test_variant']
                if test_variant != 'Control':
                    cpio_ppol_value = row['cpio/ppol']
                    self.logger.info("CPIO/PPOL VALUE: "+ str(cpio_ppol_value))
                    self.logger.info("PPO_CTRL:" + str(ppo_ctrl))
                    roi = round(cpio_ppol_value / ppo_ctrl,2)

                    df_total.loc[df_total['KPI'] =='ROI', test_variant] = roi

        elif test_obj == 'Orders':
            df_total = pd.concat([df_total, roi_row], ignore_index=True)
            for index, row in df_cpio_ppol.iterrows():
                test_variant = row['test_variant']
                if test_variant != 'Control':
                    cpio_ppol_value = row['cpio/ppol']
                    self.logger.info("CPIO/PPOL VALUE: "+ str(cpio_ppol_value))
                    self.logger.info("PPO_CTRL:" + str(ppo_ctrl))
                    roi = round(ppo_ctrl/cpio_ppol_value,2)

                    df_total.loc[df_total['KPI'] =='ROI', test_variant] = roi


        self.store.df_total_kpis = df_total
        self.logger.info(df_total)
        return df_total

    def generate_treatment_message(self) -> str :
        treatment = self.store.treatment_scope
        message = f"All values based on Treatment Scope = {treatment}"
        return message


    def generate_granularity_totals_table(self)  :
        """
        Invoked by:

        Function:
        """
        self.logger.info("GENERATE_GRANULARITY_TOTALS TABLE INVOKED")
        available_kpis = TOTAL_KPIS
        granularity_col_names = GRANULARITY_COL_NAMES

        treatment_scope = self.store.report_parameters_dict.treatment_scope

        if treatment_scope == "True":
            df = self.store.df_test_data[self.store.df_test_data['is_in_treatment']==True]

        else:
            df = self.store.df_test_data

        selected_granularity = self.store.report_parameters_dict.selected_granularity
        granularity_groups = self.store.report_parameters_dict.granularity_groups

        destination_column = granularity_col_names.get(selected_granularity)

        df_orders_granularity = pd.DataFrame
        df_profits_granularity = pd.DataFrame
        df_revenue_granularity = pd.DataFrame

        if not granularity_groups:

            if selected_granularity in ['Zone','City','Target Groups','Customer Location']:
                df_orders_granularity = self.generate_granularity_pivot_table(
                    df = df
                    , index_column= destination_column
                    , row_to_column= 'test_variant'
                    , value ='platform_order_code'
                    , aggfunc= 'count'
                )

                df_profits_granularity = self.generate_granularity_pivot_table(
                    df = df
                    , index_column= destination_column
                    , row_to_column= 'test_variant'
                    , value = 'profit_local'
                    , aggfunc='sum'
                )

                df_revenue_granularity = self.generate_granularity_pivot_table(
                    df = df
                    , index_column= destination_column
                    , row_to_column= 'test_variant'
                    , value = 'revenue_local'
                    , aggfunc= 'sum'
                )


            elif selected_granularity == 'Time of Day':
                pass

        elif granularity_groups:

            if selected_granularity in ['Zone', 'City','Target Groups','Customer Location']:
                df_with_groupings,group_columns = self.generate_granularity_grouping_col(df, destination_column)
                self.logger.info(df_with_groupings.head())

                df_orders_granularity = self.get_grouped_granularity_pivot_table(
                    df_with_groupings
                    ,group_column = group_columns
                    ,row_to_column = 'test_variant'
                    , value = 'platform_order_code'
                    , aggfunc = 'count')

                df_profits_granularity = self.get_grouped_granularity_pivot_table(
                    df_with_groupings
                    , group_column = group_columns
                    , row_to_column = 'test_variant'
                    , value = 'profit_local'
                    , aggfunc='sum'
                )

                df_revenue_granularity = self.get_grouped_granularity_pivot_table(
                    df_with_groupings,
                    group_column = group_columns
                    , row_to_column = 'test_variant'
                    , value = 'revenue_local'
                    , aggfunc = 'sum'
                )

            elif selected_granularity == 'Time of Day':
                pass

        return df_orders_granularity, df_profits_granularity, df_revenue_granularity


    def generate_granularity_grouping_col(self, df, ref_col):
        """
        Invoked by:

        Function:
        """
        granularity_groups = self.store.report_parameters_dict.granularity_groups
        added_cols = []
        for group in granularity_groups:
            group_name = group['group_name']
            df[group_name] = np.NAN
            added_cols.append(group_name)
            groupings = set(g.strip() for g in group['groupings'])  # Converting to a set reduces time complexity from O(n) to O(1)
            inc_criteria = group['inc_criteria']

            df.loc[df[ref_col].isnull(), group_name] = 0

            df[ref_col] = df[ref_col].astype(str).str.strip()

            if inc_criteria == 'in':
                df.loc[df[ref_col].notnull(), group_name] = df.loc[df[ref_col].notnull(), ref_col].apply(lambda x: 1 if x in groupings else 0)
            elif inc_criteria == 'not_in':
                df.loc[df[ref_col].notnull(), group_name] = df.loc[df[ref_col].notnull(), ref_col].apply(lambda x: 0 if x in groupings else 1)

            self.logger.info("Completed for Group: "+ group_name)

        self.store.df_data_with_groupings = df
        return df, added_cols

    def generate_granularity_pivot_table(self, df, index_column, row_to_column, value, aggfunc):
        """
        Invoked by:

        Function:
        """

        self.logger.info("Generate Granularity Pivot")
        df_pivot_granularity = df.pivot_table(index = index_column, columns = row_to_column, values = value, aggfunc = aggfunc)
        #self.logger.info(df_pivot_granularity)
        df_pivot_granularity = df_pivot_granularity.dropna()
        for col in df_pivot_granularity.columns:
            df_pivot_granularity[col] = df_pivot_granularity[col].astype(int)

        for col in df_pivot_granularity.columns:
            if col != 'Control':
                df_pivot_granularity['Delta ' + col] = df_pivot_granularity[col] - df_pivot_granularity['Control']
                df_pivot_granularity['% Delta '+ col] = 100*df_pivot_granularity['Delta '+col]/df_pivot_granularity['Control'].abs()

        for col in df_pivot_granularity.columns[1:]:
            if '% Delta' in col:

                df_pivot_granularity[col] = df_pivot_granularity[col].round(2).astype(str) + '%'
            elif '% Delta' not in col:
                df_pivot_granularity[col] = df_pivot_granularity[col].astype(int).apply(lambda x: '{:,}'.format(x))

        df_pivot_granularity.reset_index(inplace=True)
        df_pivot_granularity.rename(columns={'index': 'Granularity'}, inplace=True)
        self.logger.info(df_pivot_granularity)
        return df_pivot_granularity

    def get_delta_kpis(self):
        """
        Invoked by:

        Function:
        """
        # df = self.store.df_total_kpis

        # delta_columns = df.columns[df.columns.str.startswith('% Delta')]
        # orders_row = df[df['KPI'] =='Orders']
        # profits_row = df[df['KPI']=='Profit']

        # orders_delta = orders_row[delta_columns]
        # profits_delta = profits_row[delta_columns]

        # delta_orders = orders_delta.values.tolist()[0]
        # delta_ord = [float(i[:-1]) for i in delta_orders]
        # delta_profits = profits_delta.values.tolist()[0]
        # delta_pro = [float(i[:-1]) for i in delta_profits]
        # return delta_ord, delta_pro

        df = self.store.df_significance_data

        df_deltas = df[df['kpi_label'].isin(['profit_local_per_user','orders_per_user'])]
        df_deltas_filter = df_deltas[(df_deltas['variant_a']=='Control') & (df_deltas['treatment']=="All")]
        df_deltas_filter['delta'] = 100 * (df_deltas_filter['mean_b'] - df_deltas_filter['mean_a']) / abs(df_deltas_filter['mean_a'])
        self.logger.info(df_deltas_filter)

        orders_rows = df_deltas_filter[df_deltas_filter['kpi_label'] == 'orders_per_user']
        profits_row = df_deltas_filter[df_deltas_filter['kpi_label'] == 'profit_local_per_user']

        orders_per_user_deltas = orders_rows['delta'].to_list()
        orders_per_user_p_vals = orders_rows['p_value'].to_list()
        profit_per_user_deltas = profits_row['delta'].to_list()
        profit_per_user_p_vals = profits_row['p_value'].to_list()


        orders_per_user_deltas = [round(value, 2) for value in orders_per_user_deltas]
        profit_per_user_deltas = [round(value, 2) for value in profit_per_user_deltas]

        return orders_per_user_deltas, profit_per_user_deltas , orders_per_user_p_vals, profit_per_user_p_vals

    def get_grouped_granularity_pivot_table(self, df,group_column, row_to_column,value, aggfunc):
        """
        Invoke by:

        Returns:
        """

        df_pivot = pd.DataFrame()

        for column_group in group_column:
            filtered_df = df[df[column_group]==1]
            pivot = pd.pivot_table(filtered_df, values = value, columns  = row_to_column, aggfunc=aggfunc)
            pivot['Group'] = column_group
            pivot = pivot[['Group'] + [col for col in pivot.columns if col != 'Group']] # Ensuring Group becomes the first column
            pivot.drop(columns = ['test_variant', 'index'], errors = 'ignore', inplace = True)
            self.logger.info(pivot)
            df_pivot = pd.concat([df_pivot,pivot])

        df_pivot = df_pivot.dropna()
        for col in df_pivot.columns:
            if col not in ['Group']:
                df_pivot[col] = df_pivot[col].astype(int)

        for col in df_pivot.columns:
            if col not in ['Control','Group']:
                df_pivot['Delta ' + col] = df_pivot[col] - df_pivot['Control']
                df_pivot['% Delta '+ col] = 100*df_pivot['Delta '+col]/df_pivot['Control'].abs()

        for col in df_pivot.columns[1:]:
            if '% Delta' in col:
                df_pivot[col] = df_pivot[col].round(2).astype(str) + '%'
            elif '% Delta' not in col:
                df_pivot[col] = df_pivot[col].astype(int).apply(lambda x: '{:,}'.format(x))
        self.logger.info(df_pivot)
        return df_pivot


    def get_significance_table(self,scope,variation_a,variation_b):

        """
        Invoked by:

        Function:
        """
        df_base = self.store.df_significance_data
        df_significance = df_base[(df_base['variant_a']==variation_a) & (df_base['variant_b']==variation_b)]
        df_significance = df_significance[df_significance['treatment']==scope]
        df_significance = df_significance[['kpi_label','method','mean_a','mean_b','p_value','values_count_a','values_count_b']]
        #df_significance['is_cuped_applied'] = df_significance['is_cuped_applied'].astype(str)
        df_significance['delta'] = 100 * (df_significance['mean_b'] - df_significance['mean_a']) / abs(df_significance['mean_a'])
        df_significance[['mean_a', 'mean_b','delta']] = df_significance[['mean_a', 'mean_b','delta']].apply(lambda x: round(x, 2))
        df_significance['p_value'] = df_significance['p_value'].round(3)
        df_significance['delta'] = df_significance['delta'].astype(str) + '%'

        df_significance = df_significance[[
            'kpi_label',
            'method',
            'mean_a',
            'mean_b',
            'delta',
            'p_value',
            'values_count_a',
            'values_count_b',
            ]]
        df_significance = df_significance.rename(columns={'mean_a': variation_a
                                                          , 'mean_b': variation_b
                                                          , 'values_count_a': f'values_count_{variation_a}'
                                                          , 'values_count_b': f'values_count_{variation_b}' })

        df_significance['kpi_label'] =pd.Categorical(df_significance['kpi_label'], categories= SIG_KPIS.keys(), ordered = True)
        df_significance.sort_values(by = 'kpi_label', inplace=True)
        df_significance['kpi_label'] = df_significance['kpi_label'].map(SIG_KPIS)

        df_significance[f'values_count_{variation_a}'] = df_significance[f'values_count_{variation_a}'].astype(int).apply(lambda x: '{:,}'.format(x))
        df_significance[f'values_count_{variation_b}'] = df_significance[f'values_count_{variation_b}'].astype(int).apply(lambda x: '{:,}'.format(x))

        return df_significance

    def create_order_dist_total_graph(self, treatment_scope= "All"):
        """
        Invoked by:

        Function:
        """
        if treatment_scope == "True":
            df = self.store.df_test_data[self.store.df_test_data['is_in_treatment']==True]

        else:
            df = self.store.df_test_data

        df_filtered = self.filter_outliers_based_on_dist(df)

        variants = df['test_variant'].unique().tolist()
        variants.sort()

        colors = COLORS
        min_dist = df_filtered['tt_dist_km'].min()
        max_dist = df_filtered['tt_dist_km'].max()
        bin_size = (max_dist - min_dist) / 20  # Approximate bin size for 20 bins

        fig = go.Figure()

        for i, variant in enumerate(variants):
            var_color = colors[i % len(colors)]
            var_data = df_filtered[df_filtered['test_variant'] == variant]['tt_dist_km']
            total_volume = len(var_data)

            fig.add_trace(go.Histogram(x=var_data, xbins=dict(start=min_dist, end=max_dist, size=bin_size), name=variant, marker_color = var_color))

            # Create a KDE estimate and add it as a scatter plot
            x_kde = np.linspace(min_dist, max_dist, 1000)
            kde = gaussian_kde(var_data)
            y_kde = kde(x_kde) * bin_size * total_volume  # Adjust KDE output
            fig.add_trace(go.Scatter(x=x_kde, y=y_kde, mode='lines', name=f'{variant} KDE', line = dict(color = var_color)))


        fig.update_layout(
            autosize=False,
            width=1280,
            height=960,
            barmode='overlay',
            xaxis_title='Delivery Distance (km)',
            yaxis_title='Order Volume',
            legend = dict(
                yanchor = "top"
                , y = -0.1
                , xanchor = 'center'
                , x = 0.5
                , orientation = 'h'
            )
        )
        fig.update_traces(opacity=0.3)

        return fig

    def filter_outliers_based_on_dist(self, df):
        """
        Invoked by:

        Function:
        """
        perc_99 = df['tt_dist_km'].quantile(0.99)
        df_filtered = df[df['tt_dist_km'] <= perc_99]
        return df_filtered

    def create_granularity_graphs_for_groupings(self, treatment = "All"):
        """
        Invoked by:

        Function:
        """
        granularity_groups = self.store.report_parameters_dict.granularity_groups
        selected_granularity = self.store.report_parameters_dict.selected_granularity
        figures = []
        if not granularity_groups:
            df = self.store.df_test_data
            if treatment == "True":
                df = df[df['is_in_treatment'] == True]

            destination_col = GRANULARITY_COL_NAMES.get(selected_granularity)
            variants = df['test_variant'].unique().tolist()
            variants.sort()
            if selected_granularity in ['Zone','City','Target Groups','Customer Location']:
                unique_gran = df[destination_col].unique().tolist()
                unique_gran = [gran for gran in unique_gran if gran is not None]
                self.logger.info(str(unique_gran))
                for gran in unique_gran:
                    self.logger.info("Granularity: "+ gran)
                    df_gran = df[df[destination_col]==gran]
                    df_gran_filtered = self.filter_outliers_based_on_dist(df_gran)
                    fig = self.draw_granularity_order_dist_chart(df_gran_filtered, gran, variants)
                    figures.append(fig)
        else:
            df = self.store.df_data_with_groupings
            if treatment == "True":
                df  = df[df['is_in_treatment'] == True]

            variants = df['test_variant'].unique().tolist()
            variants.sort()
            #grouping_cols = []
            for group in granularity_groups:
                group_name = group['group_name']
                df_gran = df[df[group_name] == 1]
                df_gran_filtered = self.filter_outliers_based_on_dist(df_gran)
                fig = self.draw_granularity_order_dist_chart(df_gran_filtered,group_name, variants)
                figures.append(fig)
                self.logger.info("Graph drawn for :"+ group_name)

        return figures

    def draw_granularity_order_dist_chart(self, df,gran, variants):
        colors = COLORS
        min_dist = df['tt_dist_km'].min()
        max_dist = df['tt_dist_km'].max()
        bin_size = (max_dist - min_dist) / 20  # Approximate bin size for 20 bins

        fig = go.Figure()

        for i, variant in enumerate(variants):
            var_color = colors[i % len(colors)]
            var_data = df[df['test_variant'] == variant]['tt_dist_km']
            total_volume = len(var_data)

            # Continue only if var_data has more than one element
            if len(var_data) > 1:

                fig.add_trace(go.Histogram(x=var_data, xbins=dict(start=min_dist, end=max_dist, size=bin_size), name=variant, marker_color = var_color))

                # Create a KDE estimate and add it as a scatter plot
                x_kde = np.linspace(min_dist, max_dist, 1000)
                kde = gaussian_kde(var_data)
                y_kde = kde(x_kde) * bin_size * total_volume  # Adjust KDE output
                fig.add_trace(go.Scatter(x=x_kde, y=y_kde, mode='lines', name=f'{variant} KDE', line = dict(color = var_color)))
            else:
                self.logger.info(f"Skipping {variant} due to insufficient data.")

        fig.update_layout(
            autosize=False,
            width=750,
            height=700,
            barmode='overlay',
            xaxis_title='Delivery Distance (km)',
            yaxis_title='Order Volume',
            title={
                'text': gran,
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': dict(
                    size=16,
                    color="black",
                    family="Outfit bold, sans-serif"
                )
            }
            , legend = dict(
                yanchor = "top"
                , y = -0.1
                , xanchor = 'center'
                , x = 0.5
                , orientation = 'h'
            )
        )
        fig.update_traces(opacity=0.3)

        return fig


    ############### PRICING ACTION STANDARDS

    def get_pac_header(self, variation):

        df_sig = self.store.df_significance_data
        df_pas = df_sig[df_sig['kpi_label'].isin(['profit_local_per_user','orders_per_user','profit_local'])]
        df_pas_filtered = df_pas[(df_pas['variant_a'] == 'Control') & (df_pas['treatment'] == 'All') & (df_pas['variant_b']==variation)]
        df_profit_local = df_pas_filtered[df_pas_filtered['kpi_label']=='profit_local']
        #### Old Code for profit per order and total orders
        #df_pac = df_sig[(df_sig['kpi_label']=='profit_per_user')& (df_sig['variant_a']=='Control')& (df_sig['treatment']=='All') & (df_sig['variant_b']==variation)]
        #self.logger.info(df_pas_filtered)
        self.store.df_pac = df_profit_local

        flow :str
        #Control Metrics
        orders_per_user_control = df_pas_filtered[df_pas_filtered['kpi_label'] == 'orders_per_user']['mean_a'].iloc[0]
        profit_per_user_control = df_pas_filtered[df_pas_filtered['kpi_label'] =='profit_local_per_user']['mean_a'].iloc[0]

        #Variant Metrics
        orders_per_user_variant = df_pas_filtered[df_pas_filtered['kpi_label'] == 'orders_per_user']['mean_b'].iloc[0]
        profit_per_user_variant = df_pas_filtered[df_pas_filtered['kpi_label'] == 'profit_local_per_user']['mean_b'].iloc[0]

        # CONDITION: Order Increased Profit Decreased - Decreasing Fees flow
        if (orders_per_user_control < orders_per_user_variant) & (profit_per_user_control > profit_per_user_variant):

            flow = 'decrease_fees'
            header = 'What measures can we develop for decreasing fees?'

        # CONDITION: Order Decreased Profit Increased - Increasing Fees Flow
        elif (orders_per_user_control > orders_per_user_variant) & (profit_per_user_control < profit_per_user_variant):

            flow = 'increase_fees'
            header = 'What measures can we develop for increasing fees?'

        # CONDITION: Profit and Orders both decreased - Lose Lose Situation
        elif (orders_per_user_control > orders_per_user_variant) & (profit_per_user_control>profit_per_user_variant):

            flow = 'lose_lose'
            header = 'Profit per User and Orders per User both decreased. This means that we are losing both orders and profit and should NOT be implemented. '

        # CONDITION: Profit and Orders both increased this is a GOLDEN TEST.
        elif (orders_per_user_control < orders_per_user_variant) & (profit_per_user_control < profit_per_user_variant):

            flow = 'win_win'
            header = 'Profit per User and Orders per User both increased. If the key KPIs are significant then test should be implemented. '

        # CONDITION: Catch edge cases where either orders or profit returned the exact same value or did not return a valid response due to check being too early.
        else:

            flow = 'error'
            header = 'Error! Values not present. Please ensure test has been deployed for at least 7 days.'

        self.store.pas_flow = flow
        return header

    def get_pac_checks(self,variation):

        flow = self.store.pas_flow
        self.logger.info("FLOW: "+ flow)

        if flow == "increase_fees":
            return *PAC_CHECKS_INCREASE,

        elif flow == "decrease_fees":
            return *PAC_CHECKS_DECREASE,

        elif flow == "win_win":
            return ["","","",""]

        elif flow == "lose_lose":
            return ["","","",""]



    def get_pac_theories(self,variation):

        flow = self.store.pas_flow
        if flow == "increase_fees":
            return *PAC_THEORY_INCREASE,
        elif flow == "decrease_fees":
            return *PAC_THEORY_DECREASE,
        elif flow == "win_win":
            return "","","",""
        elif flow == "lose_lose":
            return "","","",""

    def get_pac_comparisons(self, variation):
        flow = self.store.pas_flow
        if flow == "increase_fees":
            return *PAC_COMPARE_INCREASE,
        elif flow == "decrease_fees":
            return *PAC_COMPARE_DECREASE,
        elif flow =="win_win":
            return "","","",""
        elif flow == "lose_lose":
            return "","","",""

    def get_pac_measures(self, variation):

        flow = self.store.pas_flow
        df_pac = self.store.df_pac
        curr_cd = self.store.curr_cd
        self.logger.info("Currency for PAS: "+ curr_cd)
        self.logger.info(df_pac)

        if curr_cd == EUR:
            curr_conv = self.store.curr_conv_ratio
        else:
            curr_conv = 1



        #ON CALCULATING THE FIELDS: Wherever the PAS has 'Diff in.....', it means it is an absolute. We are not looking at subtraction but the difference betwen the two and the difference is always positive.
        #To-Do: Move orders to adjusted orders.

        df_pac['cpio/ppol_variant_b'] = (abs(df_pac['mean_b']- df_pac['mean_a'])*df_pac['values_count_b'])/abs(df_pac['values_count_b']-df_pac['values_count_a'])
        df_pac['accepted_order_diff']= 100*(abs(df_pac['mean_b']-df_pac['mean_a'])*df_pac['values_count_b'])/(df_pac['mean_a']*df_pac['values_count_a'])
        df_pac['delta_orders'] = 100*abs(df_pac['values_count_b']-df_pac['values_count_a'])/df_pac['values_count_a']

        target_ppo = self.store.target_ppo

        # WHAT MEASURES CAN BE USED FOR INCREASING FEES?
        if flow == "increase_fees":

            df_pac['is_cond_1_satisfied'] = df_pac.apply(lambda row: row['cpio/ppol_variant_b'] > row['mean_a'], axis=1)

            df_pac['accepted_roi'] = 50
            df_pac['recoup_roi'] = 100*(df_pac['mean_a']*abs(df_pac['values_count_b']- df_pac['values_count_a']))/(abs(df_pac['mean_b']-df_pac['mean_a'])*df_pac['values_count_b'])

            df_pac['is_cond_2_satisfied'] = df_pac.apply(lambda row: True if (row['delta_orders'])< (row['accepted_order_diff']) else False, axis = 1)
            df_pac['is_cond_3_satisfied'] = df_pac.apply(lambda row: True if row['recoup_roi']<row['accepted_roi'] else False, axis = 1)
            df_pac['is_cond_4_satisfied'] = df_pac.apply(lambda row: True if (row['cpio/ppol_variant_b']) > target_ppo else False, axis = 1)

            for index, row in df_pac.iterrows():

                if row['is_cond_1_satisfied']:
                    step_1_measure = str(round(curr_conv*row['cpio/ppol_variant_b'],2))+ "  >  " +str(round(curr_conv*row['mean_b'],2))
                    check_1 = 1
                else:
                    step_1_measure = str(round(curr_conv*row['cpio/ppol_variant_b'],2))+ "  <  " +str(curr_conv*round(row['mean_b'],2))
                    check_1 = 0

                if row['is_cond_2_satisfied']:
                    step_2_measure = str(round(row['delta_orders'],2))+ "%  <  " + str(round(row['accepted_order_diff'],2)) +"%"
                    check_2 = 1
                else:
                    step_2_measure = str(round(row['delta_orders'],2))+ "%  >  " + str(round(row['accepted_order_diff'],2)) + "%"
                    check_2 = 0

                if row['is_cond_3_satisfied']:
                    step_3_measure = str(round(row['recoup_roi'],2))+ "%  <  " + str(round(row['accepted_roi'],2)) + "%"
                    check_3 = 1
                else:
                    step_3_measure = str(round(row['recoup_roi'],2))+ "%  >  " + str(round(row['accepted_roi'],2)) + "%"
                    check_3 = 0

                if row['is_cond_4_satisfied']:
                    step_4_measure = "N/A"
                    check_4 = 2
                else:
                    step_4_measure = "N/A"
                    check_4 = 2

            return step_1_measure, step_2_measure, step_3_measure, step_4_measure, check_1,check_2, check_3, check_4


        # WHAT MEASURES CAN BE USED FOR DECREASING FEES?
        elif flow == "decrease_fees":

            df_pac['is_cond_1_satisfied'] = df_pac.apply(lambda row: (row['cpio/ppol_variant_b']) < row['mean_a'], axis =1)
            df_pac['is_cond_2_satisfied'] = df_pac.apply(lambda row: True if (row['delta_orders']) > (row['accepted_order_diff']) else False, axis=1)
            df_pac['is_cond_4_satisfied'] = df_pac.apply(lambda row: True if (row['cpio/ppol_variant_b']) < target_ppo else False, axis = 1)

            for index, row in df_pac.iterrows():

                if row['is_cond_1_satisfied']:
                    step_1_measure = str(round(curr_conv*row['cpio/ppol_variant_b'],2)) + " < " + str(round(curr_conv*row['mean_b'],2))
                    check_1 = 1
                else:
                    step_1_measure = str(round(curr_conv*row['cpio/ppol_variant_b'],2)) + " > " + str(round(curr_conv*row['mean_b'],2))
                    check_1 = 0

                if row['is_cond_2_satisfied']:
                    step_2_measure = str(round(row['delta_orders'],2)) + "%  >  "  + str(round(row['accepted_order_diff'],2)) + "%"
                    check_2= 1
                else:
                    step_2_measure = str(round(row['delta_orders'],2)) + "% <  " + str(round(row['accepted_order_diff'],2)) + "%"
                    check_2 = 0

                step_3_measure = "Visit Marketing Dashboard"
                check_3 = 2

                if row['is_cond_4_satisfied']:
                    step_4_measure = 'N/A'
                    check_4 = 2
                else:
                    step_4_measure = 'N/A'
                    check_4 = 2

            return step_1_measure, step_2_measure, step_3_measure, step_4_measure, check_1, check_2, check_3, check_4

        else :
            return "","","","",np.nan, np.nan, np.nan, np.nan

### frontend.py

In [ ]:

"""
----------INPUT COMPONENTS----------
The functions below return HTML, CSS components that are displayed on the app. Each HTML div or in some cases are treated as separate functions that can be invoked based on the state of
the app. This allows the displays to be dynamic rather than just a static input
"""

# Dropdown component to generate the entities available to the user.
def get_filter_entity_dropdown(results_tool_adapter:ResultsToolAdapter) ->html.Div:
    """
    Returns the dropdown for the filter entity
    """
    return (
        html.Div(children=[
                html.Label('Entity'),
                dcc.Dropdown(options=results_tool_adapter.store.fetch_entities()
                             , placeholder='Select entity'
                             , id="results-tool-entities-dropdown-component"
                             , style = {'fontFamily':'Outfit, sans-serif'}
                             )
                ],
                id='results-tool-entities-dropdown'
            )
    )

# Dropdown component to generate the tests available to the user.
def get_filter_test_dropdown() -> html.Div:
    """
    Returns the button component used for triggering
    """
    return (
        html.Div(children= [
            html.Label('Test Name'),
            dcc.Dropdown(options=[]
                         ,placeholder= "Select test"
                         ,id= "results-tool-test-dropdown-component"
                         , style = {'fontFamily':'Outfit, sans-serif'}
                         )
        ],
        id= 'results-tool-test-dropdown')
    )

# Load Test Data Button
def get_load_test_data_button():
    """
     Returns the button component used for triggering loading of test data
    """

    return (dbc.Button('Load Test'
        , n_clicks=0
        , id='results-tool-load-test-data-button'
        , className="btn btn-primary mt-4"
        , style={'backgroundColor': '#131732'
                 , 'fontFamily':'Outfit, sans-serif'})
        )

def get_reset_button():
    return(
        html.A(
            dbc.Button("Reset"
                       , id = 'results-tool-reset-button'
                       , className="btn btn-primary mt-4"
                       , style = {
                           'backgroundColor':'#D61F26'
                           , 'fontFamily':'Outfit, sans-serif'
                       })
            , href = "/"
        )
    )

def get_spacing_row(height = '20px'):
        """
        Returns a blank row of 20 pixels height. This can be called in the layout whenever you require some spacing in a row for a cleaner aesthetic. Future improvement would make the spacing
        dynamic.
        """
        return(dbc.Row(style={'height': height})
)

def get_grouping_controls(results_tool_adapter:ResultsToolAdapter) -> html.Div:
    """
    HTML Div that contains all the elements and components that allows the user to interact with the grouping options to analyze the test.
    """
    return (
        dcc.Loading(
            id="loading-component",
            type="circle",
            children=[
                html.Div([
                    dbc.Row(
                        dbc.Col(id="analysis-option-header"
                        , children="Analysis Options"
                        , style = {
                            'fontWeight':'600'
                            , 'fontFamily':'Outfit, sans-serif'
                            }
                        )
                    ),
                    get_spacing_row(),
                    dbc.Row(
                        [dbc.Col(id="treatment-selector-component",
                            children=[
                            html.Label('Treatment scope'),
                            dbc.RadioItems(
                                options=[
                                    {'label': 'True', 'value': 'True'},
                                    {'label': 'All', 'value': 'All'}
                                ],
                                value='All',
                                inline=True,
                                style={'color': '#131732', 'paddingLeft': 10, 'marginRight':'40px', 'fontFamily':'Outfit, sans-serif'},
                                id='treatment-selector'
                                )
                            ],
                            width= 4
                        ),
                        dbc.Col(
                            [
                                dbc.Row(
                                    [
                                        dbc.Col(
                                            children="Exclusions"
                                            , width = 8
                                        )
                                    ]

                                ),
                                dbc.Row(
                                    [
                                        dbc.Col(
                                            dbc.Checklist(
                                                options = [
                                                    {
                                                        "label": "Subscribers",
                                                        "value": "sub"
                                                    }
                                                ]
                                                , id = 'subscriber-exclusion'
                                            ),
                                            width = 6,
                                        ),
                                        dbc.Col(
                                            dbc.Checklist(
                                                options = [
                                                    {
                                                        "label":"FDNC",
                                                        "value": "fdnc"
                                                    }
                                                ]
                                                , id = 'fdnc-exclusion'
                                            ),
                                            width = 6,
                                        )
                                    ]
                                )
                            ],
                            width = 5
                        )
                        , dbc.Col([
                            html.Div(children=[
                                html.Label('Currency', style = {'fontFamily':'Outfit, sans-serif'}),
                                dcc.Dropdown(options=[]
                                             , value = 'EUR'
                                            , id="results-tool-currency-dropdown-component"
                                            , style = {'fontFamily':'Outfit, sans-serif'}
                                            )
                            ],
                                id='results-tool-currency-dropdown'
                            ),
                            dbc.Tooltip(
                                "Significance dashboard will be always be in local currency."
                                , target = 'results-tool-currency-dropdown'
                            )
                        ])
                        ]
                    ),
                    get_spacing_row(),
                    dbc.Row(
                        [
                            dbc.Col(html.Div(children=[
                                html.Label('Group Data by'
                                        , style = {'fontFamily':'Outfit, sans-serif'}),
                                dcc.Dropdown(options=[]
                                            , value = results_tool_adapter.store.no_granularity
                                            , placeholder='Select granularity'
                                            , id="results-tool-grouping-dropdown-component")
                                            ],
                                            id='results-tool-grouping-dropdown'
                                            )
                                    ,width = 6
                            ),
                            dbc.Col(
                                html.Div(children=[
                                    html.Label('Granularity Groups'),
                                        dcc.Dropdown(options= []
                                                    , placeholder = 'Groups'
                                                    , id = 'results-tool-data-group-count-dropdown-component'
                                                    )
                                        ]
                                        ,id = 'results-tool-data-group-count-dropdown'
                                        , style = {'display': 'none'}
                                )
                                ,width = 3
                            ),
                            dbc.Col(
                                dbc.Button('Load Groups'
                                , n_clicks=0
                                , id='results-tool-load-data-groups-button'
                                , className="btn btn-primary mt-4"
                                , style={'backgroundColor': '#131732', 'display':'none', 'fontFamily':'Outfit, sans-serif'}
                                , disabled= True)
                                )
                        ]
                    ),
                    get_spacing_row(),
                    dbc.Row(
                        dbc.Col(
                            html.Div(id = 'dynamic-grouping-input-container')
                            ,width=12
                            , style= {'display':'block'}
                        )
                    )
                ],
                id = 'results-tool-analysis-options-component',
                style = {'display':'none'}
                )
            ]
        )
    )
def get_loading_spinner() -> html.Div:
    return(html.Div(
        dcc.Loading(
            id = 'loading',
            type = 'circle',
            children = html.Div(id = 'loading-output-component')
        )
        )
    )

def get_grouping_input_field(group_count) -> html.Div:
    return(html.Div(
            [html.Div([
                    dbc.Row([
                        dbc.Col([
                            html.Label('Group Name', style={'display': 'none' if i != 0 else 'block', 'fontSize':'11px'}),
                            dbc.Input(type='text', id={'type':'group-name','index':i}, placeholder=f'Group Name {i+1}')
                        ], width=3, style={'marginBottom': '10px'
                                           , 'fontFamily':'Outfit, sans-serif'}),
                        dbc.Col([
                            html.Label('Inclusion', style={'display': 'none' if i != 0 else 'block', 'fontSize':'11px'}),
                            dcc.Dropdown(id={'type':'in-not-in', 'index':i}, options=[{'label': 'is', 'value': 'in'}, {'label': 'is not in', 'value': 'not_in'}], value='in', style ={'backgroundColor':'#F5F5F6', 'fontSize':'10px'})
                        ], width=2, style={'marginBottom': '10px'
                                           , 'fontFamily':'Outfit, sans-serif'}),
                        dbc.Col([
                            html.Label('Options', style={'display': 'none' if i != 0 else 'block', 'fontSize':'11px'}),
                            dcc.Dropdown(id={'type':'grouping-options-dropdown', 'index':i}, options=[], multi=True)
                        ], width=7, style={'marginBottom': '10px'
                                           , 'fontFamily':'Outfit, sans-serif'})
                    ], id=f'grouping-option-row-{i}')  # Add the id here
                ]) for i in range(int(group_count))
            ]
        )
    )

def get_groups_loaded_message(granularity) -> html.Div:
    message = f'All {granularity} loaded to be analyzed individually. '
    return(html.Div(
                dbc.Row(
                    dbc.Alert(message, color = 'success')
                )
            )
    )

def get_grouping_overview() -> html.Div:
    """
    """
    return(
        html.Div(
                 dbc.Row(
                     html.A(
                        children =[
                            html.I(className='bi bi-input-cursor')
                            , html.Span (' Give Feedback')
                        ]
                        , href = 'https://forms.gle/srsPWh7AUYf5Byju5'
                        , target = '_blank'
                        , style={
                            'fontSize':'16px'
                            , 'fontFamily': 'Outfit, sans-serif'
                        }
                    )
                )
                ,  id = 'results-tool-analysis-overview-component'
                , style ={'display':'none'}
        )
    )

def get_report_buttons() -> html.Div:
    """
    Is displayed when Load Test button is clicked. It displays 3 buttons:
    1. Create Report
    2. Download Report
    3. Download Data
    """
    return (html.Div(
                dbc.Row([
                    dbc.Col(
                        dbc.Button('Create Report', id = 'generate-report-button', className = 'btn-block'
                                   , style ={
                                       'fontSize':'12px'
                                       , 'backgroundColor': '#4629FF'
                                       , 'border':'1px solid #4629FF'
                                       , 'fontFamily':'Outfit, sans-serif'
                                   },
                                   disabled = False)
                        , width = 3
                        ,className = 'mx-auto'
                    )
                    , dbc.Col(
                        dbc.Button('Download Report', id = 'download-report-button', className = 'btn-block'
                                   , style ={
                                       'fontSize':'12px'
                                       , 'backgroundColor':'#D61F26'
                                       , 'border':'1px solid #D61F26'
                                       , 'fontFamily':'Outfit, sans-serif'
                                   },
                                   disabled = True)
                        , width = 3
                        ,className = 'mx-auto'
                    )
                    , dbc.Col(
                        dbc.Button('Download Data', id = 'download-data-button', className = 'btn-block'
                                   , style ={
                                       'fontSize':'12px'
                                       , 'backgroundColor': '#131732'
                                       , 'fontFamily':'Outfit, sans-serif'
                                   })
                        , width = 3
                        ,className = 'mx-auto'
                    )
                ], justify='center', style = {'padding': '10px'})
                , id = 'generate-button-layout-component'
                , style= {'display':'none'}
            )
    )

"""
----------------------------------------------------------------REPORT COMPONENTS----------------------------------------------------------------
"""
def get_report(results_tool_adapter: ResultsToolAdapter) -> html.Div:

    report_spacing = '10px'

    heading_div = html.Div(
        children= "test_name",
        id= 'report-heading-component',
        style = {
            'border': '2px solid'
            ,'textAlign': 'center'  # Center the text
            ,'backgroundColor': '#131732'
            , 'color': '#FFFFFF' # Set the text color to white
            , 'fontWeight':'bold'
            , 'fontSize':'20px'
            , 'borderRadius':'5px'
            , 'borderColor':'#0E122E'
            , 'fontFamily':'Outfit, sans-serif'
        }
    )

    performance_overview_component = html.Div(
        [
            dbc.Row(
                html.Div(
                    children = 'Performance Overview'
                    , id = 'performance-overview-header-component'
                    , style = {
                        'fontWeight':'bold'
                        , 'fontSize':'16px'
                    }
                )
            )
            , get_spacing_row(10)
            , dbc.Row(
                [
                    dbc.Col(
                        children = "Variants"
                        , id = 'performance-overview-variant-component'
                        , width = 3
                    )
                    , dbc.Col(
                        children = "Delta Profit per User"
                        , id = 'performance-overview-profit-deltas-component'
                        , width =3
                    )
                    , dbc.Col(
                        children = "Delta Orders per User"
                        , id = 'performance-overview-order-deltas-component'
                        , width = 3
                    )
                    , dbc.Col(
                        children = "CPiO/PPoL"
                        , id = 'performance-overview-roi-component'
                        , width = 3
                    )
                ]
            )
            , dbc.Row(
                [
                    dbc.Col(
                        children = "Variants"
                        , id = 'performance-overview-variant-values-component'
                        , width = 3
                    )
                    , dbc.Col(
                        children = "Profit Deltas"
                        , id = 'performance-overview-profit-deltas-values-component'
                        , width =3
                    )
                    , dbc.Col(
                        children = "Order Deltas"
                        , id = 'performance-overview-order-deltas-values-component'
                        , width = 3
                    )
                    , dbc.Col(
                        children = "CPiO/PPoL"
                        , id = 'performance-overview-roi-values-component'
                        , width = 3
                    )
                ]
            )
            , get_spacing_row(20)
            , dbc.Row(
                dbc.Col(children = "Arrows indicate directional change and are not representative of significance. Please consult significance table."
                        , style ={
                            'fontSize': '10px'
                        })
            )
        ]
        , style = {
             'border': '1px solid'
            ,'textAlign': 'center'  # Center the text
            ,'backgroundColor': '#F5F5F6' # Set the text color to white
            , 'borderRadius':'5px'
            , 'borderColor':'#C5C5C9'
            , 'fontFamily':'Outfit, sans-serif'

        }
    )

    test_overview_div = html.Div(
        [
            dbc.Row(
                dbc.Col(
                    children = 'Test Summary'
                    , id = 'test-summary-heading-component'
                    , style = {
                            'fontWeight':'600'
                            , 'fontSize': '16px'
                            , 'textAlign':'center'
                        }
                )
            )
            , get_spacing_row(10)
            , dbc.Row(
                [
                    dbc.Col(
                        children = 'Objective: '
                        , id = 'test-objective-component'
                        , width = 4
                        , style = {
                            'fontWeight':'600'
                        }
                    )
                    , dbc.Col(
                        children = 'Test Objective from DPS'
                        , id = 'test-objective-value-component'
                        , width = 8
                    )
                ]
            )
            , dbc.Row(
                [
                    dbc.Col(
                        children = 'Hypothesis: '
                        , id = 'test-hypothesis-component'
                        , width = 4
                        , style = {
                            'fontWeight':'600'
                        }
                    )
                    , dbc.Col(
                        children = 'Test Hypothesis from DPS'
                        , id = 'test-hypothesis-value-component'
                        , width = 8
                    )
                ]
            )
            , dbc.Row(
                [
                    dbc.Col(
                        children = 'User Split'
                        , id = 'user-split-component'
                        , width = 4
                        , style = {
                            'fontWeight':'600'
                        }
                    )
                    , dbc.Col(
                        children = 'Variant'
                        , id = 'user-split-variation-component'
                        , width = 4

                    )
                    , dbc.Col(
                        children = 'Split'
                        , id = 'user-split-value-component'
                        , width = 4
                    )
                ]
            )
            , dbc.Row(
                [
                    dbc.Col(
                        children = 'Start Date'
                        , id = 'start-date-component'
                        , width = 4
                    )
                    , dbc.Col(
                        children = 'End Date'
                        , id = 'end-date-component'
                        , width = 4
                    )
                ], style={
                    'fontWeight':'600'
                }
            )
            , dbc.Row(
                [
                    dbc.Col(
                        children = 'Start Date Value'
                        , id = 'start-date-value-component'
                        , width = 4
                    )
                    , dbc.Col(
                        children = 'End Date Value'
                        , id = 'end-date-value-component'
                        , width = 4
                    )
                ]
            )
        ]
        , style = {'border': '1 px solid '
                    ,'backgroundColor':'#A2FAA3'
                    ,'borderRadius':'5px'
                    , 'borderColor':'#63CF65'
                    , 'fontFamily':'Outfit, sans-serif'
            }

    )

    power_tool_message = dbc.Alert("Please ensure that the test had enough power. ", color="warning")

    pricing_action_standards = html.Div(
        [
            dbc.Row(
                dbc.Col([
                    html.H5('Pricing Action Standards',
                            style={'textAlign': 'center', 'fontFamily': 'Outfit, sans-serif', 'display': 'inline-block'}),
                    dbc.Button(
                        html.I(className="bi bi-info-circle-fill"),
                        href="https://docs.google.com/presentation/d/10kQ-Kta-HZLFnAMdQvh6QJjW7GrYBDDm-8H_fKZwE1w/edit?pli=1#slide=id.g10d2cd51ad5_2_10",
                        target="_blank",
                        color="grey",
                        style={'marginLeft': '10px', 'verticalAlign': 'middle', 'padding': '0'}
                    )
                ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'})
            )
            , get_spacing_row('10px')
            , dbc.Row(
                [
                    dbc.Col(
                        "Select the variant you want to compare against Control. "
                        ,width = 7
                    )
                    , dbc.Col(
                        children = dcc.Dropdown(
                                        options = []
                                        , placeholder =  'Select Variant'
                                        , id ='pac-variation-selector'
                                        , style = {'backgroundColor':'#F5F5F6'}
                                        )
                                    , width = 3
                    )
                ]
            )
            , get_spacing_row('10px')
            , dbc.Row(
                children = ""
                , id = 'pac-header-message'
            )
            , get_spacing_row('10px')
            , dbc.Row(
                [
                    dbc.Col(""
                            , id = 'pac-check-header'
                            , width = 4
                            , style = {'fontWeight':'bold', 'textAlign':'left', 'padding':'0px'})
                    , dbc.Col(""
                              ,id= 'pac-theory-header'
                              , width = 3
                              , style = {'fontWeight':'bold'})
                    , dbc.Col(""
                              , id = 'pac-comparison-header'
                              , width = 2
                              , style = {'fontWeight':'bold'})
                    , dbc.Col(""
                              , id = 'pac-measure-header'
                              , width = 2
                              , style = {'fontWeight':'bold'})
                    , dbc.Col(""
                              , id = 'pac-result-header'
                              ,width = 1
                              , style = {'fontWeight':'bold'})
                ]
            )
            , get_spacing_row('10px')
            , dbc.Row(
                [
                    dbc.Col(
                        ""
                        , width = 4
                        , id = 'pac-step-1-check'
                        , style = {'textAlign':'left', 'padding':'0px'}
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-1-theory'
                        ,width = 3
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-1-comparison'
                        , width = 2
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-1-measure'
                        , width = 2
                    )
                    , dbc.Col(
                        children = ""
                        , id = 'pac-step-1-result'
                        , width = 1
                    )
                ]
                , style={
                    'fontSize':'12px'
                    ,'fontFamily':'Outfit, sans-serif'
                }
            )
            , get_spacing_row('10px')
            , dbc.Row(
                [
                    dbc.Col(
                        ""
                        , width = 4
                        , id = 'pac-step-2-check'
                        , style = {'textAlign':'left', 'padding':'0px'}
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-2-theory'
                        ,width = 3
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-2-comparison'
                        , width = 2
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-2-measure'
                        , width = 2
                    )
                    , dbc.Col(
                        children = ""
                        , id = 'pac-step-2-result'
                        , width = 1
                    )
                ]
                , style={
                    'fontSize':'12px'
                    ,'fontFamily':'Outfit, sans-serif'
                }
            )
            , get_spacing_row('10px')
            , dbc.Row(
                [
                    dbc.Col(
                        ""
                        , width = 4
                        , id = 'pac-step-3-check'
                        , style = {'textAlign':'left', 'padding':'0px'}
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-3-theory'
                        ,width = 3
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-3-comparison'
                        , width = 2
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-3-measure'
                        , width = 2
                    )
                    , dbc.Col(
                        children = ""
                        , id = 'pac-step-3-result'
                        , width = 1
                    )
                ]
                , style={
                    'fontSize':'12px'
                    ,'fontFamily':'Outfit, sans-serif'
                }
            )
            , get_spacing_row('10px')
            , dbc.Row(
                [
                    dbc.Col(
                        ""
                        , width = 4
                        , id = 'pac-step-4-check'
                        , style = {'textAlign':'left', 'padding':'0px'}
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-4-theory'
                        ,width = 3
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-4-comparison'
                        , width = 2
                    )
                    , dbc.Col(
                        ""
                        , id = 'pac-step-4-measure'
                        , width = 2
                    )
                    , dbc.Col(
                        children = ""
                        , id = 'pac-step-4-result'
                        , width = 1
                    )
                ]
                , style={
                    'fontSize':'12px'
                    ,'fontFamily':'Outfit, sans-serif'
                }
            )
            , dbc.Row(
                dbc.Col(
                    html.P([
                        "Pricing Action Standards aims to identify the most efficient and effective pricing solution while being easily comparable with other solutions. It can be an effective way of benchmarking pricing experiments against other marketing initiatives. ",
                        html.A("Read more about it here.", href="https://docs.google.com/presentation/d/10kQ-Kta-HZLFnAMdQvh6QJjW7GrYBDDm-8H_fKZwE1w/edit", target="_blank")
                    ], id='pac-generic-message', style={'fontWeight': 'bold', 'textAlign': 'center', 'padding': '0px', 'display': 'block'})
                )
            )
        ]
        , style = {
             'border': '1px solid'
            ,'textAlign': 'center'  # Center the text
            ,'backgroundColor': '#FFFFFF' # Set the background color to white
            , 'borderRadius':'5px'
            , 'borderColor':'#C5C5C9'
            , 'fontFamily':'Outfit, sans-serif'
            , 'padding':'40px'

        }
    )

    significance_kpis = html.Div(
        [
            html.H5('KPIs Significance'
                    , style = {'textAlign':'center'
                                , 'fontFamily':'Outfit, sans-serif'
                                })
            , dbc.Tabs(
                    [
                        dbc.Tab(label = 'All'
                                , tab_id = "All"
                                , children=[
                                    html.Div(
                                        [
                                            get_spacing_row()
                                            , dbc.Row(
                                                [
                                                    dbc.Col(
                                                        width = 1
                                                    )
                                                    , dbc.Col(
                                                        children = dcc.Dropdown(
                                                            options = []
                                                            , placeholder =  'Select Variant'
                                                            , id ='variation-a-all-dropdown-component'
                                                            , style = {'backgroundColor':'#F5F5F6'}
                                                        )
                                                        , width = 3
                                                    )
                                                    , dbc.Col(
                                                        children = dcc.Dropdown(
                                                            options = []
                                                            , placeholder =  'Select Variant'
                                                            , id ='variation-b-all-dropdown-component'
                                                            , style = {'backgroundColor':'#F5F5F6'}
                                                        )
                                                        , width = 3
                                                    )
                                                    , dbc.Col(
                                                        children = [
                                                                    html.Label("Alpha")
                                                                    , dcc.Slider(min = 0
                                                                              , max = 0.1
                                                                              , step = 0.025
                                                                              , value = 0.05
                                                                              , marks = None
                                                                              , tooltip={"placement":"bottom","always_visible":True}
                                                                              , id = 'significant-all-slider')
                                                        ], width = 4
                                                    )
                                                    , dbc.Col(
                                                        width = 1
                                                    )
                                                ]
                                            )
                                            , get_spacing_row()
                                            , dbc.Row(
                                                [
                                                    dbc.Col(width = 1)
                                                    , dbc.Col(
                                                        children=""
                                                        , id = 'significance-table-all-component'
                                                        , width = 10
                                                    )
                                                    , dbc.Col(width = 1)
                                                ]
                                            )
                                            , get_spacing_row()
                                            , dbc.Row(
                                                children = ""
                                                , id = 'significance-table-currency-all-component'
                                                , style = {
                                                    'fontSize':'11px'
                                                    ,'fontFamily':'Outfit, sans-serif'
                                                }
                                            )
                                        ]

                                    )
                                ])
                        , dbc.Tab(label = 'True'
                                  , tab_id= "True"
                                  , children=[
                                        html.Div(
                                            [
                                                get_spacing_row()
                                                , dbc.Row(
                                                    [
                                                        dbc.Col(
                                                            width = 1
                                                        )
                                                        , dbc.Col(
                                                            children = dcc.Dropdown(
                                                                options = []
                                                                , placeholder =  'Select Variant'
                                                                , id ='variation-a-true-dropdown-component'
                                                                , style = {'backgroundColor':'#F5F5F6'}
                                                            )
                                                            , width = 3
                                                        )
                                                        , dbc.Col(
                                                            children = dcc.Dropdown(
                                                                options = []
                                                                , placeholder =  'Select Variant'
                                                                , id ='variation-b-true-dropdown-component'
                                                                , style = {'backgroundColor':'#F5F5F6'}
                                                            )
                                                            , width = 3
                                                        )
                                                        , dbc.Col(
                                                                children = [
                                                                    html.Label("Alpha")
                                                                    , dcc.Slider(min = 0
                                                                              , max = 0.1
                                                                              , step = 0.025
                                                                              , value = 0.05
                                                                              , marks = None
                                                                              , tooltip={"placement":"bottom","always_visible":True}
                                                                              , id = 'significant-true-slider')
                                                        ], width = 4
                                                    )
                                                        , dbc.Col(
                                                            width = 1
                                                        )
                                                    ]
                                                )
                                                , get_spacing_row()
                                                , dbc.Row(
                                                    [
                                                        dbc.Col(width = 1)
                                                        , dbc.Col(
                                                            children=""
                                                            , id = 'significance-table-true-component'
                                                            , width = 10
                                                        )
                                                        , dbc.Col(width = 1)
                                                    ]

                                                )
                                                , get_spacing_row()
                                                , dbc.Row(
                                                    children = ""
                                                    , id = 'significance-table-currency-true-component'
                                                    , style = {
                                                        'fontSize':'11px'
                                                        ,'fontFamily':'Outfit, sans-serif'
                                                    }
                                                )
                                            ]

                                        )
                                    ]
                                )
                    ]
                    , id = 'significance-card-tabs'
                    , active_tab = 'All'
                )
            ]
            , id = 'significance-card'
            , style = {
                'border': '1px solid '
                ,'backgroundColor':'#F5F5F6'
                ,'borderRadius':'5px'
                , 'borderColor':'#C5C5C9'
                , 'fontFamily':'Outfit, sans-serif'
            }
    )

    test_totals = html.Div(
        dbc.Container(
            [
                dbc.Row(children="Gross Metrics Overview"
                        , id = 'report-totals-header-component'
                        , style = {
                            'fontWeight': '600'
                            ,'fontSize': '18px'
                            , 'justifyContent':'center'
                            , 'fontFamily':'Outfit, sans-serif'
                            , 'textAlign':'center'
                        } )
                , dbc.Row(children = "Test Totals Component"
                        , id = 'report-totals-component'
                        , style = {
                            'fontFamily':'Outfit, sans-serif'
                        }
                )
                , dbc.Row(children = "Treatment Scope Message"
                        , id = 'report-totals-treatment-message-component'
                        , className = 'mt-auto'
                        , style = {
                            'fontFamily':'Outfit, sans-serif'
                            ,'fontSize': '12px'
                            , 'textAlign':'right'
                        }
                )
                , dbc.Row(children = "Currency Message"
                          , id  = 'report-totals-currency-message-component'
                          , className = 'mt-auto'
                          , style = {
                              'fontFamily':'Outfit, sans-serif'
                              , 'fontSize':'12px'
                          })
            ]
            ,style = {
                    'border': '1px solid'
                    , 'backgroundColor': '#F5F5F6'
                    , 'borderRadius':'5px'
                    , 'borderColor':'#C5C5C9'
                    , 'fontFamily':'Outfit, sans-serif'
                    , 'display': 'flex'
                    , 'flexDirection': 'column'
                    }
        )
    )


    test_setup = html.Div(
        dbc.Col(
            children= "Test Setup Component"
            , id = 'test-setup-component'
            , style = {
                'border': '1px solid'
                , 'borderRadius':'5px'
                , 'borderColor':'#C5C5C9'
                , 'backgroundColor':'#F5F5F6'
                , 'fontFamily':'Outfit, sans-serif'
                , 'display':'none'
                }
        )
    )

    order_dist_graphs = html.Div(
        [
            dbc.Row(
            html.H5('Order Distribution by Distance'
                    , style = {'textAlign':'center'
                                , 'fontFamily':'Outfit, sans-serif'
                            })
        )
        , dbc.Tabs(
            [
                dbc.Tab(label = "All"
                        , tab_id = "All"
                        , children = [
                            html.Div(
                                [
                                    get_spacing_row()
                                    , dbc.Row(
                                        dcc.Graph(
                                            id ='order-distribution-total-plotly-all'
                                            , figure = {}
                                        )
                                        , id = 'order-distribution-graph-card-all'
                                    )
                                ]
                            )
                        ]

                )
                , dbc.Tab(label = "True"
                          , tab_id = "True"
                          , children = [
                              html.Div(
                                  [
                                      get_spacing_row()
                                      , dbc.Row(
                                          dcc.Graph(
                                              id = 'order-distribution-total-plotly-true'
                                              , figure = {}
                                          )
                                          , id = 'order-distribution-graph-card-true'
                                      )
                                  ]
                              )
                          ]
                )
            ]
        )

        ]
    )

    granularity_totals = html.Div(
        dbc.Card(
            [
                dbc.CardHeader(
                    [
                        html.H6('Total KPIs on Chosen Granularity'
                                , style = {'textAlign':'center'
                                            , 'fontFamily':'Outfit, sans-serif'
                                            })
                        , dbc.Tabs(
                            [
                                dbc.Tab(label = 'Orders'
                                        , tab_id = "Orders"
                                        , children = [
                                            html.Div(
                                                [
                                                    get_spacing_row()
                                                    , dbc.Row("Table", id ='orders-tab-granularity-component')
                                                ]

                                            )
                                        ]
                                        , style = {'fontFamily':'Outfit, sans-serif'})
                                , dbc.Tab(label = 'Profit'
                                          , tab_id= "Profit"
                                          , children = [
                                                html.Div(
                                                    [
                                                        get_spacing_row()
                                                        , dbc.Row("Table", id ='profits-tab-granularity-component')
                                                    ]

                                                )
                                        ]
                                          , style = {'fontFamily':'Outfit, sans-serif'})
                                , dbc.Tab(label = 'Revenue'
                                          , tab_id= "Revenue"
                                          , children =  [
                                                html.Div(
                                                    [
                                                        get_spacing_row()
                                                        , dbc.Row("Table", id ='revenue-tab-granularity-component')
                                                    ]

                                                )
                                            ]
                                          , style={'fontFamily':'Outfit, sans-serif'})
                            ]
                            , id = 'granularity-tabs-component'
                            , active_tab = 'Orders'
                        )
                    ]
                    , id = 'granularity-totals-card-header'
                    , style = {'backgroundColor':'#F5F5F6'}
                )
            ], style = {'padding':'10px'}
        ), id = 'granularity-totals-component'
        , style = {'display':'none'}
    )

    granularity_dist_graphs = html.Div(
        [
            dbc.Row(
            html.H5('Order Distribution by Distance'
                    , style = {'textAlign':'center'
                                , 'fontFamily':'Outfit, sans-serif'
                            })
            )
            , dbc.Tabs(
                [
                    dbc.Tab(
                        label="All"
                        , tab_id= "All"
                        , children = [
                            get_spacing_row()
                            , dbc.Row(
                                dbc.Container(
                                    children = ""
                                    , id = 'order-dist-granularity-all-container'
                                )
                            )
                        ]
                    )
                    , dbc.Tab(
                        label="True"
                        , tab_id = "True"
                        , children = [
                            get_spacing_row()
                            , dbc.Row(
                                dbc.Container(
                                    children = ""
                                    , id = 'order-dist-granularity-true-container'
                                )
                            )
                        ]
                    )
                ]
            )

        ]
        ,id="granularity-order-distribution-card-component"
        , style = {'display':'none'}

    )

    return (
        dcc.Loading(
            id="loading",
            type="circle",
            children=[
                html.Div(
                    [
                        dbc.Row(heading_div)
                        , dbc.Row(get_spacing_row(report_spacing))
                        , dbc.Row(
                            [
                                dbc.Col(test_overview_div, width = 5)
                                , dbc.Col(
                                    [
                                        dbc.Row(performance_overview_component)
                                        , get_spacing_row(10)
                                        , dbc.Row(power_tool_message)
                                    ]
                                    , width = 7
                                )
                            ]
                        )
                        , dbc.Row(get_spacing_row(10))
                        , dbc.Row(pricing_action_standards)
                        , dbc.Row(get_spacing_row(report_spacing))
                        , dbc.Row(significance_kpis)
                        , dbc.Row(get_spacing_row(report_spacing))
                        , dbc.Row(test_totals)
                        , dbc.Row(get_spacing_row(report_spacing))
                        , dbc.Row(test_setup)
                        , dbc.Row(get_spacing_row(report_spacing))
                        , dbc.Row(order_dist_graphs)
                        , dbc.Row(get_spacing_row(report_spacing))
                        , dbc.Row(granularity_totals)
                        , dbc.Row(get_spacing_row(report_spacing))
                        , dbc.Row(granularity_dist_graphs)
                    ]
                    , id = 'complete-report-component'
                    , style = {'display':'none'}
                )
            ]
        )
    )

def get_roi_values(roi_dict, curr_cd):

    roi_values = []
    for key,value in roi_dict.items():
        roi_values.append(
            dbc.Row([
                dbc.Col(html.Span(f"{value} {curr_cd}", style = {'fontSize':'14px'
                                                                 , 'fontFamily':'Outfit, sans-serif'}))
            ]
            , justify = 'center'
            , align='center'
             )
        )
    return roi_values

def get_variants(roi_dict):

    variants = []
    for key, value in roi_dict.items():
        variants.append(
            dbc.Row([
                dbc.Col(html.Span(f"{key}", style = {'fontSize':'14px'
                                                     , 'fontFamily':'Outfit, sans-serif'}))
            ])
        )

    return variants

def get_delta_values(values_list,sig_values):

    delta_values = []

    for value,sig in zip(values_list,sig_values):

        icon = ""
        color = ""

        #NO CHANGE
        if value == 0:
            icon = "bi-dash-circle-fill"
            color = "grey"

        # Small increase significant
        elif (0 < value <= 2) & (sig <= 0.05):
            icon = "bi-arrow-up-right-circle-fill"
            color = "#00D11F"

        # Small increase not significant
        elif (0 < value <= 2) & (sig > 0.05):
            icon = "bi-arrow-up-right-circle-fill"
            color = "grey"

        # Small decrease significant
        elif (0 > value >= -2) & (sig <= 0.05):
            icon = "bi-arrow-down-right-circle-fill"
            color = "#D61F26"

        # Small decrease non-significant
        elif (0 > value >= -2) & (sig >0.05):
            icon = 'bi-arrow-down-right-circle-fill'
            color = 'grey'

        # Big increase significant
        elif (value > 2) & (sig <= 0.05):
            icon = "bi-arrow-up-circle-fill"
            color = "#00D11F"

        # Big increase insignificant
        elif (value > 2) & (sig > 0.05):
            icon = "bi-arrow-up-circle-fill"
            color = "grey"

        # Big decrease significant
        elif (value < -2) & (sig <0.05):
            icon = "bi-arrow-down-circle-fill"
            color = "#D61F26"

        # Big decrease insignificant
        elif (value < -2) & (sig > 0.05):
            icon = 'bi-arrow-down-circle-fill'
            color = 'grey'

        delta_values.append(
            dbc.Row([
                dbc.Col([
                    html.Span(f"{value} %  ", style = {'fontSize':'14px','fontFamily':'Outfit, sans-serif'}),
                    html.I(className=icon, style={'color': color, 'fontSize': '14px'}),
                ]),
            ], justify='center', align='center')
        )

    return delta_values

def get_pas_results(check):

    if check == 1:
        icon = "bi-check-circle-fill"
        color = "#00D11F"
        icon_component = html.I(className = icon, style = {'color': color, 'fontSize': '18px'})
    elif check == 0:
        icon = "bi-x-circle-fill"
        color = "#D61F26"
        icon_component = html.I(className = icon, style = {'color': color, 'fontSize': '18px'})
    elif check == 2:
        icon = "bi-exclamation-circle-fill"
        color ="grey"
        icon_component = html.I(className = icon, style = {'color': color, 'fontSize': '18px'})
    else:
        icon_component = ""


    return icon_component


def get_variation_in_overview(var_list):

    var_col = []
    for var in var_list:
        var_col.append(
            dbc.Row([
                dbc.Col(html.Span(f"{var}", style = {'fontSize':'14px'
                                                     , 'fontFamily':'Outfit, sans-serif'}))
            ])
        )
    return var_col


def create_grid_graph(order_dist_gran_all):

    total_graphs = len(order_dist_gran_all)
    total_rows = (total_graphs + 1) // 2
    rows = []

    counter = 0

    for row in range(total_rows):
        row_content = []
        for col in range(2):
            if counter < total_graphs:
                graph = order_dist_gran_all[counter]

                row_content.append(
                    dbc.Col(
                        dcc.Graph(
                            id = f'graph-{row}-{col}'
                            , figure = graph
                        )
                        , width = 6
                    )
                )
                counter += 1

        rows.append(dbc.Row(row_content))

    grid = html.Div(rows)

    return grid

"""
What does a Callback function do?
Callbacks are a decorator. Whenever the input property changes the function that the '@app.callback' wraps around gets called and updated.
Each callback will have at least an input and an output. The Input would be answer - when should the function get called? and the output should
answer - where should the return of the function be shown?
"""
def callbacks(app:Dash, results_tool_adapter:ResultsToolAdapter):
    """
    Occurance: When Entity ID is selected.
    Function: Updates Tests Dropdown
    """
    @app.callback(
        Output(component_id="results-tool-test-dropdown-component", component_property="options")
        , Input(component_id="results-tool-entities-dropdown-component",component_property="value")
        , prevent_initial_call = True
    )
    def update_tests_dropdown(entity_id:str):
        return results_tool_adapter.store.fetch_tests(entity_id)


    """
    Occurance: When Load Test button is clicked.
    Function: Load test granularities that are available, load test order data, display Analysis Options, Display analysis overview, Display buttons
    """
    @app.callback(
        Output(component_id='results-tool-analysis-options-component',component_property='style')
        , Output(component_id='results-tool-analysis-overview-component',component_property='style')
        , Output(component_id='generate-button-layout-component', component_property='style')
        , Output(component_id='results-tool-currency-dropdown-component', component_property='options')
        , Output(component_id='results-tool-grouping-dropdown-component',component_property='options')
        , Input(component_id='results-tool-load-test-data-button', component_property = 'n_clicks')
        , State(component_id='results-tool-entities-dropdown-component', component_property = 'value')
        , State(component_id='results-tool-test-dropdown-component', component_property='value')
        , prevent_initial_call = True
    )
    def toggle_display_analysis_option(
        n_clicks:int,
        selected_entity:str,
        selected_test:str,

        ):

        query_inputs = TestQueryInputs(
            entity_id_filter= selected_entity,
            test_name_filter= selected_test
        )

        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.submit(results_tool_adapter.load_test_groupings_into_store,query_inputs)
            executor.submit(results_tool_adapter.load_test_data_into_store,query_inputs)
            executor.submit(results_tool_adapter.load_currency_code_into_store, query_inputs)
            executor.submit(results_tool_adapter.load_significance_data_into_store, query_inputs)


        results_tool_adapter.load_target_ppo(query_inputs)

        if n_clicks is None or n_clicks==0:
            return {'display': 'none'}, {'display':'none'}, {'display':'none'}
        else:
            style = {'display':'block'}
            style_buttons = {'display':'block', 'border':'2px dashed #F5F5F6', 'borderRadius':'5px', 'backgroundColor':'#F5F5F6'}
            available_currencies = results_tool_adapter.store.curr_cd_list
            available_granularities = results_tool_adapter.store.available_granularities
            results_tool_adapter.create_report_parameter()

            return style,style, style_buttons,available_currencies, available_granularities

    """
    Occurance:
    Function:
    """
    @app.callback(
        Output(component_id='dummy-output', component_property='children')
        , Input(component_id='treatment-selector', component_property='value')
        , prevent_initial_call = True
    )
    def update_treatment_scope(treatment_sc):
        if treatment_sc:
            results_tool_adapter.store.treatment_scope = treatment_sc
            results_tool_adapter.create_report_parameter()
        return no_update

    """
    Occurance:
    Function:
    """
    @app.callback(
        Output(component_id = 'dummy-output-2', component_property = 'children')
        , Input(component_id = 'subscriber-exclusion', component_property = 'value')
        , Input(component_id = 'fdnc-exclusion', component_property = 'value')
        , prevent_initial_call = True
    )
    def update_exclusions(subscribers, fdnc):

        results_tool_adapter.update_orders_with_exclusions(subscribers, fdnc)

        return no_update

    """
    Occurance:
    Function:
    """
    @app.callback(
        Output(component_id = 'dummy-output-3', component_property = 'children')
        , Input(component_id= 'results-tool-currency-dropdown-component', component_property= 'value')
        , prevent_initial_call = True
    )
    def update_currency_code(selected_curr):
        results_tool_adapter.update_selected_currency(selected_curr)
        return no_update

    """
    Generate the number of data groups that the analyst can select.
    """
    @app.callback(
        Output(component_id='results-tool-data-group-count-dropdown-component', component_property = 'options')
        , Output(component_id='results-tool-data-group-count-dropdown', component_property='style')
        , Output(component_id='results-tool-load-data-groups-button', component_property = 'style')
        , Input(component_id='results-tool-grouping-dropdown-component', component_property= 'value')
        , prevent_initial_call = True
    )
    def generate_viable_grouping_counts(chosen_granularity):
        if(chosen_granularity == results_tool_adapter.store.no_granularity):
            results_tool_adapter.store.chosen_granularity = chosen_granularity
            results_tool_adapter.create_report_parameter()
            return [], {'display': 'none'}, {'display': 'none'}
        else:
            results_tool_adapter.store.chosen_granularity = chosen_granularity
            results_tool_adapter.generate_data_group_count_options(chosen_granularity)
            results_tool_adapter.create_report_parameter()
            return results_tool_adapter.store.available_data_groupings, {}, {'fontSize':'12px','fontFamily':'Outfit, sans-serif','backgroundColor': '#131732'}

    """
    Occurance:
    Function:
    """
    @app.callback(
        Output(component_id='results-tool-load-data-groups-button', component_property='disabled')
        , Output(component_id='dynamic-grouping-input-container',component_property='children')
        , Input(component_id='results-tool-data-group-count-dropdown-component',component_property='value')
        , prevent_initial_call = True
    )
    def enable_load_groups_button(group_count):
        max_count = len(results_tool_adapter.store.available_data_groupings)+1
        results_tool_adapter.store.selected_group_count = group_count
        if group_count < max_count:
            return False, get_grouping_input_field(group_count)
        else:
            return True, get_groups_loaded_message(results_tool_adapter.store.chosen_granularity)

    """
    Occurance:
    Function:
    """
    @app.callback(
        Output(component_id={'type':'grouping-options-dropdown','index':MATCH},component_property='options'),
        Input(component_id='results-tool-load-data-groups-button',component_property='n_clicks'),
        prevent_initial_call = True
    )
    def generate_input_fields(n_clicks):
        group_count = results_tool_adapter.store.selected_group_count
        chosen_granularity = results_tool_adapter.store.chosen_granularity
        if n_clicks == 0 or n_clicks == None:
            return []
        else :
            return results_tool_adapter.populate_granularity_options_into_input_dropdowns(chosen_granularity)

    @app.callback(
        Output('generate-report-button', 'disabled'),
        [Input({'type': 'group-name', 'index': ALL}, 'value'),
        Input({'type': 'in-not-in', 'index': ALL}, 'value'),
        Input({'type': 'grouping-options-dropdown', 'index': ALL}, 'value')]
        , prevent_initial_call = True
    )
    def enable_button(group_names, in_not_in_values, dropdown_values):
    # Check if all inputs and dropdowns have a value
        all_inputs = group_names + in_not_in_values + dropdown_values
            # If all inputs and dropdowns have a value, enable the button
        if None not in all_inputs:
            group_parameters_list = []

            for i in range(len(group_names)):

                group_params = GroupParameters(group_names[i], in_not_in_values[i], dropdown_values[i])
                group_parameters_list.append(group_params.group_parameter_dict())

            results_tool_adapter.create_report_parameter(group_parameters_list)
            return False
        else:
            # If any input or dropdown doesn't have a value, disable the button
            return True


    @app.callback(
        Output(component_id ='complete-report-component',component_property='style')                                 # Show/Hide Report
        , Output(component_id = 'granularity-totals-component', component_property='style')                          # Show/Hide granularity totals component
        , Output(component_id = 'granularity-order-distribution-card-component', component_property='style')          # Show/Hide granularity order distribution component
        , Output(component_id = 'report-heading-component', component_property='children')                          # Display test name in the report heading
        , Output(component_id = 'test-objective-value-component', component_property='children')
        , Output(component_id = 'test-hypothesis-value-component', component_property='children')
        , Output(component_id = 'user-split-variation-component', component_property='children')
        , Output(component_id = 'user-split-value-component', component_property='children')
        , Output(component_id = 'start-date-value-component', component_property='children')
        , Output(component_id = 'end-date-value-component', component_property='children')
        , Output(component_id = 'performance-overview-variant-values-component', component_property='children')
        , Output(component_id = 'performance-overview-profit-deltas-values-component', component_property='children')
        , Output(component_id = 'performance-overview-order-deltas-values-component', component_property='children')
        , Output(component_id = 'performance-overview-roi-values-component', component_property='children')
        , Output(component_id = 'performance-overview-roi-component', component_property ='children')
        , Output(component_id = 'report-totals-component', component_property = 'children')                         # Show test totals table
        , Output(component_id = 'report-totals-treatment-message-component', component_property= 'children')
        , Output(component_id = 'report-totals-currency-message-component', component_property= 'children')        # Show the treatment scope message in the test totals component
        , Output(component_id = 'orders-tab-granularity-component', component_property= 'children')
        , Output(component_id = 'profits-tab-granularity-component', component_property= 'children')
        , Output(component_id = 'revenue-tab-granularity-component', component_property= 'children')
        , Output(component_id = 'variation-a-all-dropdown-component', component_property= 'options')
        , Output(component_id = 'variation-b-all-dropdown-component', component_property= 'options')
        , Output(component_id = 'variation-a-true-dropdown-component', component_property='options')
        , Output(component_id = 'variation-b-true-dropdown-component', component_property = 'options')
        , Output(component_id = 'significance-table-currency-all-component', component_property = 'children')
        , Output(component_id = 'significance-table-currency-true-component', component_property = 'children')
        , Output(component_id = 'order-distribution-total-plotly-all', component_property= 'figure')
        , Output(component_id = 'order-distribution-total-plotly-true', component_property= 'figure')
        , Output(component_id= 'order-dist-granularity-all-container', component_property= 'children')
        , Output(component_id= 'order-dist-granularity-true-container', component_property= 'children')
        , Output(component_id= 'pac-variation-selector', component_property= 'options')
        , Input(component_id='generate-report-button', component_property = 'n_clicks')
        , prevent_initial_call = True
    )
    def create_report(n_clicks):

        """
        Returns:
            1. Style - Toggle display of whole report
            2. Style - Toggle display of totals granularity
            3. Style - Toggle display of order distribution granularity
            4. Children - Test name
            5. Children - Test objective in summary card
            6. Children - Test Hypothesis in summary card
            7. Children - User split variations in summary card
            8. Children - user split values in summary card
            9. Children - start date values in summary card
            10. Children - end date values in summary card
            11. Children - variants values in summary card
            12. Children - profit deltas in performance overview
            13. Children - order deltas in performance overview
            14. Children - CPIO/PPoL in perforamnce overview
            15. Children - CPiO/PPoL header in performance overview
            15. Children - reports totals table
            16. Children - reports totals message
            17. Options  - Variation A in All Dropdown options
            18. Options - Variation B in All Dropdown options
            19. Options - Variation A in True Dropdown options
            20. Options - Variation B in True Dropdown options
            21. Image - Plotly graph for order distribution All
            22. Image - Plotly graph for order distribution True
        """

        # Display toggles - can be returned based on what is needed in the report
        no_display = {'display':'none'}
        display = {'display':'block'}

        if n_clicks == 0 or n_clicks == None:
            return (
                no_display
                ,no_display
                ,no_display
                ,""
                ,""
                ,""
                ,""
                ,""
                ,""
                ,""
                ,""
                ,""
                ,""
                ,""
                ,""
                ,""
                ,[]
                ,[]
                ,[]
                ,[]
                ,""
                ,""
                , ""
                , ""
                ,""
                ,""
                ,[]
            )
        else:

            results_tool_adapter.filter_exclusions()

            test_name = results_tool_adapter.store.test_query_inputs.test_name_filter


            test_objective, test_hypothesis, start_date, end_date, variation_groups, variation_share = results_tool_adapter.get_test_overview_card_data()
            variation_names = get_variation_in_overview(variation_groups)
            variation_splits = get_variation_in_overview(variation_share)


            curr_cd = results_tool_adapter.store.curr_cd
            roi_json = results_tool_adapter.get_roi_of_each_variant()
            roi_dict = json.loads(roi_json)

            variants = get_variants(roi_dict)
            results_tool_adapter.get_totals_table()

            delta_orders,delta_profits, sig_orders, sig_profits = results_tool_adapter.get_delta_kpis()
            delta_orders_values = get_delta_values(delta_orders, sig_orders)
            delta_profits_values = get_delta_values(delta_profits, sig_profits)

            roi_card_roi_values = get_roi_values(roi_dict, curr_cd)
            cpio_ppol_header = results_tool_adapter.get_cpio_ppol_header()

            totals_table_treatment_message = results_tool_adapter.generate_treatment_message()
            totals_table = dbc.Table.from_dataframe(results_tool_adapter.store.df_total_kpis
                                                    , striped = True
                                                    , bordered = True
                                                    , hover = True)

            variations_in_significance = variation_groups
            variations_pac  = [var for var in variations_in_significance if 'Var' in var]
            order_dist_total_graph = results_tool_adapter.create_order_dist_total_graph()
            order_dist_total_graph_true = results_tool_adapter.create_order_dist_total_graph("True")

            total_table_curr_message = f"Monetory KPIs in the Gross Metrics Overview table are all in '{curr_cd}'"
            significance_curr_message = f"Monetary KPIs in the significance dashboard are always in the local currency. Statistically Significant KPIs have the delta column highlighted in green"


            if(results_tool_adapter.store.chosen_granularity == results_tool_adapter.store.no_granularity):
                display_granularity_totals, display_granularity_graphs = no_display, no_display
                gran_orders_table, gran_profit_table, gran_rev_table = None, None, None
                order_dist_all_graphs, order_dist_true_graphs = None, None
            else:
                display_granularity_totals, display_granularity_graphs  = display, display
                df_gran_order, df_gran_profit, df_gran_rev = results_tool_adapter.generate_granularity_totals_table()
                gran_orders_table = dbc.Table.from_dataframe(df_gran_order
                                                             , striped = True
                                                             , bordered = True
                                                             , hover = True)
                gran_profit_table = dbc.Table.from_dataframe(df_gran_profit
                                                             , striped = True
                                                             , bordered = True
                                                             , hover = True)
                gran_rev_table = dbc.Table.from_dataframe(df_gran_rev
                                                          , striped = True
                                                          , bordered = True
                                                          , hover = True)

                order_dist_gran_all = results_tool_adapter.create_granularity_graphs_for_groupings()
                order_dist_gran_true = results_tool_adapter.create_granularity_graphs_for_groupings("True")

                order_dist_all_graphs = create_grid_graph(order_dist_gran_all)
                order_dist_true_graphs = create_grid_graph(order_dist_gran_true)


            return (display
                    , display_granularity_totals
                    , display_granularity_graphs
                    , test_name
                    , test_objective
                    , test_hypothesis
                    , variation_names
                    , variation_splits
                    , start_date
                    , end_date
                    , variants
                    , delta_profits_values
                    , delta_orders_values
                    , roi_card_roi_values
                    , cpio_ppol_header
                    , totals_table
                    , totals_table_treatment_message
                    , total_table_curr_message
                    , gran_orders_table
                    , gran_profit_table
                    , gran_rev_table
                    , variations_in_significance
                    , variations_in_significance
                    , variations_in_significance
                    , variations_in_significance
                    , significance_curr_message
                    , significance_curr_message
                    , order_dist_total_graph
                    , order_dist_total_graph_true
                    , order_dist_all_graphs
                    , order_dist_true_graphs
                    , variations_pac
            )

    @app.callback(
        Output(component_id ='significance-table-all-component', component_property = 'children')
        ,State(component_id='variation-a-all-dropdown-component', component_property='value')
        ,Input(component_id = 'significant-all-slider', component_property='value')
        ,Input(component_id='variation-b-all-dropdown-component', component_property ='value')
        ,prevent_initial_call = True
    )
    def update_significance_table_all(variation_a,alpha,variation_b):
        alpha_2 = alpha
        trigger = ctx.triggered[0]['prop_id'].split('.')[0]

        if trigger == 'variation-a-all-dropdown-component' and variation_b is None:
            raise exc.PreventUpdate
        if trigger == 'variation-b-all-dropdown-component' and variation_a is None:
            raise exc.PreventUpdate

        if variation_a == variation_b:
            sig_table = dbc.Alert("Please select two different variants", color = 'warning')
        else:
            df_significance = results_tool_adapter.get_significance_table("All",variation_a,variation_b)
            data = json.loads(df_significance.to_json(orient = 'records'))
            sig_table = dash_table.DataTable(
                id = 'significance-table-true'
                , columns = [{'name':i, "id": i} for i in df_significance.columns]
                , data = data
                , style_data = {
                    'fontFamily':'Outfit, sans-serif'
                }
                , style_header = {
                    'fontWeight':'bold'
                    , 'fontFamily':'Outfit, sans-serif'
                }
                , style_data_conditional=[
                    {

                        "if":{
                        "column_id":"delta"
                        , "filter_query":f"{{p_value}} <= {alpha_2}"
                        }
                        , "backgroundColor":"#A2FAA3"
                        , "fontWeight":"bold"
                    }

                ]
            )

        return sig_table

    @app.callback(
        Output(component_id ='significance-table-true-component', component_property = 'children')
        ,State(component_id='variation-a-true-dropdown-component', component_property='value')
        ,Input(component_id='significant-true-slider', component_property='value')
        ,Input(component_id='variation-b-true-dropdown-component', component_property ='value')
        ,prevent_initial_call = True
    )
    def update_significance_table_all(variation_a,alpha,variation_b):

        trigger = ctx.triggered[0]['prop_id'].split('.')[0]

        if trigger == 'variation-a-true-dropdown-component' and variation_b is None:
            raise exc.PreventUpdate
        if trigger == 'variation-b-true-dropdown-component' and variation_a is None:
            raise exc.PreventUpdate

        if variation_a == variation_b:
            sig_table = dbc.Alert("Please select two different variants", color = 'warning')
        else:
            df_significance = results_tool_adapter.get_significance_table("True",variation_a,variation_b)
            data = json.loads(df_significance.to_json(orient = 'records'))
            sig_table = dash_table.DataTable(
                id = 'significance-table-true'
                , columns = [{'name':i, "id": i} for i in df_significance.columns]
                , data = data
                , style_data = {
                    'fontFamily':'Outfit, sans-serif'
                }
                , style_header = {
                    'fontWeight':'bold'
                    , 'fontFamily':'Outfit, sans-serif'
                }
                , style_data_conditional=[
                    {

                        "if":{
                        "column_id":"delta"
                        , "filter_query":f"{{p_value}} <= {alpha}"
                        }
                        , "backgroundColor":"#A2FAA3"
                        , "fontWeight":"bold"
                    }

                ]
            )

        return sig_table

    @app.callback(
        Output(component_id = 'download-test-data-xlsx', component_property = 'data')
        , Input(component_id = 'download-data-button', component_property = 'n_clicks')
        , prevent_initial_call = True
    )
    def download_data_xlsx(n_clicks):
        df = results_tool_adapter.store.df_test_data
        filename = results_tool_adapter.store.test_query_inputs.test_name_filter + '_order_data.csv'
        return dcc.send_data_frame(df.to_csv, filename = filename)


    @app.callback(
        Output(component_id = 'pac-generic-message', component_property='style')
        , Output(component_id='pac-header-message', component_property = 'children')
        , Output(component_id = 'pac-check-header', component_property='children')
        , Output(component_id = 'pac-theory-header', component_property='children')
        , Output(component_id = 'pac-comparison-header', component_property = 'children')
        , Output(component_id = 'pac-measure-header', component_property='children')
        , Output(component_id='pac-step-1-check', component_property='children') #1
        , Output(component_id='pac-step-2-check', component_property = 'children') #2
        , Output(component_id='pac-step-3-check', component_property = 'children') #3
        , Output(component_id='pac-step-4-check', component_property = 'children') #4
        , Output(component_id='pac-step-1-theory', component_property='children') #1
        , Output(component_id='pac-step-2-theory', component_property = 'children') #2
        , Output(component_id='pac-step-3-theory', component_property = 'children') #3
        , Output(component_id='pac-step-4-theory', component_property = 'children') #4
        , Output(component_id = 'pac-step-1-comparison', component_property = 'children') #1
        , Output(component_id = 'pac-step-2-comparison', component_property = 'children') #2
        , Output(component_id = 'pac-step-3-comparison', component_property = 'children') #3
        , Output(component_id = 'pac-step-4-comparison', component_property = 'children') #4
        , Output(component_id='pac-step-1-measure', component_property='children') #1
        , Output(component_id='pac-step-2-measure', component_property='children') #2
        , Output(component_id='pac-step-3-measure', component_property='children') #3
        , Output(component_id='pac-step-4-measure', component_property='children') #4
        , Output(component_id = 'pac-step-1-result', component_property='children') #1
        , Output(component_id = 'pac-step-2-result', component_property='children') #2
        , Output(component_id = 'pac-step-3-result', component_property='children') #3
        , Output(component_id = 'pac-step-4-result', component_property='children') #4
        , Input(component_id='pac-variation-selector', component_property = 'value')
        , prevent_initial_call = True
    )
    def pricing_action_standards(variation):

        no_display = {'display':'none'}
        pac_header = results_tool_adapter.get_pac_header(variation)
        check = "Step"
        theory = "Theory"
        compare = "Comparison"
        measure = "Measure"


        step_1_check, step_2_check, step_3_check, step_4_check = results_tool_adapter.get_pac_checks(variation)
        step_1_theory, step_2_theory, step_3_theory, step_4_theory = results_tool_adapter.get_pac_theories(variation)
        step_1_compare, step_2_compare, step_3_compare, step_4_compare = results_tool_adapter.get_pac_comparisons(variation)
        step_1_measure,step_2_measure,step_3_measure,step_4_measure, check_1, check_2, check_3, check_4 = results_tool_adapter.get_pac_measures(variation)

        icon_step_1 = get_pas_results(check_1),
        icon_step_2 = get_pas_results(check_2),
        icon_step_3 = get_pas_results(check_3),
        icon_step_4 = get_pas_results(check_4)

        if step_1_check == step_1_theory == step_1_measure == step_1_compare == "":
            check = ""
            theory = ""
            compare = ""
            measure = ""

        return (
            no_display,
            pac_header,
            check,theory, compare, measure,
            step_1_check,
            step_2_check,
            step_3_check,
            step_4_check,
            step_1_theory,
            step_2_theory,
            step_3_theory,
            step_4_theory,
            step_1_compare,
            step_2_compare,
            step_3_compare,
            step_4_compare,
            step_1_measure,
            step_2_measure,
            step_3_measure,
            step_4_measure,
            icon_step_1,
            icon_step_2,
            icon_step_3,
            icon_step_4
        )



"""
Layout function can be considered as the HTML of the page. This holds the components that will exist in the page. This layout is grouped into nested rows and columns. Each row can contain
upto 12 columns. If there are more than 1 column in a row, it should be seen as an array of columns and hence denoted as [dbc.Col(), dbc.Col()]. The columns can be made dynamic
"""
def layout(results_tool_adapter:ResultsToolAdapter):

    return (html.Div([
                dbc.Row([
                    dbc.Col(html.H1("Automated Experiment Report", className="text-heading"
                                    , style={'color': '#FFFFFF', 'fontSize': '28px','fontWeight':'bold'})
                                    , width={"offset":0}
                                    , style={
                                        'backgroundColor': '#131732'
                                        , 'display': 'flex'
                                        , 'alignItems': 'center'
                                        , 'justifyContent': 'center'
                                        , 'fontFamily':'Outfit, sans-serif'}
                            )
                ])
                    , dbc.Row(style={'height':'15px'})
                    ,
                    dbc.Container([
                        dbc.Row([
                            dbc.Col(html.H6("Select Test to Analyze", className='test-selector-heading'
                                            , style = {'color': '#3A3B3C'
                                                       , 'fontFamily':'Outfit, sans-serif'}
                                            ))
                    ])
                    , dbc.Row([
                        dbc.Col(get_filter_entity_dropdown(results_tool_adapter),width=2)
                        , dbc.Col(get_filter_test_dropdown(), width= 6)
                        , dbc.Col(get_load_test_data_button(), width= 2)
                        , dbc.Col(get_reset_button(), width = 2)
                    ])
                    , dbc.Row(html.Hr(style = {'marginTop':'20px'
                                            ,'marginBottom':'20px'
                                            ,'borderTop':'3px solid #F5F5F6'}))
                    , dbc.Row([
                        dbc.Col(get_grouping_controls(results_tool_adapter),width=6)
                        , dbc.Col(
                            [dbc.Row(
                                dbc.Col(get_grouping_overview(), width = 12))
                            , get_spacing_row()
                            , dbc.Row(
                                dbc.Col(get_report_buttons(), width = 12)
                            )
                            , dbc.Row(
                                dcc.Download(id = 'download-test-data-xlsx')
                            )
                            ]

                        )
                    ])
                ])
                , get_spacing_row()
                , dbc.Container(
                    [
                        dbc.Row(html.Hr(style = {'marginTop':'20px'
                                            ,'marginBottom':'20px'
                                            ,'borderTop':'3px solid #F5F5F6'}))
                        , dbc.Row(
                            get_report(results_tool_adapter)
                        )
                    ]
                )
                , dbc.Row(
                    html.Div(id='dummy-output', style={'display': 'none'})
                )
                , dbc.Row(
                    html.Div(id='dummy-output-2', style={'display': 'none'})
                )
                , dbc.Row(
                    html.Div(id='dummy-output-3', style={'display': 'none'})
                )
            ]
        )
    )

### init Results Tool Module

In [ ]:
app_logger.info("Initializing Automated Experiments Results Module.....")
results_connector = ResultsToolConnector(ENV, **BIGQUERY_CONFIG)
results_adapter = ResultsToolAdapter(results_connector)

results_tool_module = TabModule(
    module_name = "Automated Experiment Report"
    , env = ENV
    , dash = app
    , layout = layout
    , callbacks = callbacks
    , adapter = results_adapter
)

results_module_tab = results_tool_module.get_tab()

2025-01-22 15:38:46,437 - INFO - Initializing Automated Experiments Results Module.....
INFO:APP:Initializing Automated Experiments Results Module.....


# Deploy Application

In [ ]:
#################################### LAUNCH APP
app_logger.info("Launching App.....")
app.layout = dbc.Container([
            dbc.Tabs([
                #pm_module_component,
                #power_module_tab,
                #sbt_module_tab,
                results_module_tab
            ])
        ],
        fluid=True
        )

app.run_server(debug=True, jupyter_mode="inline")